# Github Repo with this notebook and related files
https://github.com/austinjwang/DLH_Project

# Background [[1](#References)]
Modern machine learning algorithms can extract relevant features from medical data and make predictions for previously unseen patients. Current deep learning architectures used for risk prediction, based on Electronic Medical Records (EMR) data generally employ attention layers on top of recurrent layers. The models allow for state-of-the-art predictions and have enhanced interpretability by virtue of using Attention models. However, they fall short on processing time-series data (diagnosis codes and procedure codes in EMR) that was sampled at irregular time intervals. Predictive accuracy was comparable across neural network architectures. Groups of patients suffering from infectious complications, with chronic or progressive conditions, and for whom standard medical care was not suitable. Attention-based networks may be preferable to recurrent networks if an interpretable model is required, at only marginal cost in predictive accuracy.

# Paper explanation
The paper our project was based on proposed evaluating different deep learning architectures for predicting ICU readmission and describing patients at risk. The innovation of the method lies in its comparison of various deep learning architectures. This approach goes beyond traditional machine learning methods and explores the potential of state-of-the-art deep learning techniques in healthcare analytics. Model performance was gauged using metrics such as accuracy, precision, recall, and AUC. The effectiveness of the method would depend on how well it performed compared to the baseline models or existing clinical scoring systems in predicting ICU readmissions and providing interpretable insights into patient risk factors. The contribution of the research lies in its exploration of advanced deep learning techniques in a critical healthcare domain. By benchmarking and comparing different architectures, the paper provides valuable insights into the potential of deep learning for improving predictive accuracy and interpretability in ICU readmission prediction, thereby contributing to the ongoing research in healthcare analytics and patient care optimization.  The paper also concluded that several different models were viable with no one approach being by far the best, and choosing a model could depend on factors such as the need for interpretability of results.

# Scope of Reproducibility:

The hypothesis of the paper is below:
*   The utilization of various deep learning architectures, including LSTM, CNN, and Transformer models, will lead to improved accuracy in predicting readmission to the Intensive Care Unit (ICU) and provide valuable insights into identifying patients-at-risk compared to traditional machine learning methods or clinical scoring systems.

The goal of the project is to explore different approaches to time embeddings (MCE with time aware attention, embedding layers with concatenated elapsed times and embedding layers + neural ODE’s), across different deep learning architecture combinations. Our project is based on an existing study that concluded Neural ODEs applied to code embeddings did generally resulted in improved performance, suggesting that they may constitute a building block of interest for neural networks processing not only continuous time series, but also timestamped codes. 

We would assume that the conclusion in the study holds true and through our implementation of the models attempt to prove that out.  From a computational perspective, the size of the MIMIC III tables involved and runtime of the scripts lead us to believe that these results can be reproduced using personal computers without the need for external memory or computational power.


# Environment

* Python Version: Python 3.11+
* Required Packages (no specific version requirements):
  * pandas
  * numpy
  * tqdm
  * torch
  * torchdiffeq
  * scipy
  * sklearn
  * tabulate (displaying results at the end of this notebook only)

# Methodology

The methodology section of this notebook contains subsections to describe the appraoch for data, models, training, and testing/evaluation.

In [84]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchdiffeq import odeint, odeint_adjoint
import os
from time import time
import scipy.stats as st
from sklearn.metrics import *
import torch.utils.data as utils
import pickle
from tabulate import tabulate

from torch.utils.data.dataset import (
    ChainDataset,
    ConcatDataset,
    Dataset,
    IterableDataset,
    StackDataset,
    Subset,
    TensorDataset,
    random_split,
)

from torch.utils.data.dataloader import (
    DataLoader,
    _DatasetKind,
    get_worker_info,
    default_collate,
    default_convert,
)

import warnings
warnings.filterwarnings('ignore')

## Data
The data for this project comes from the MIMIC III Clinical Database. Specifically, we use the following tables which can be downloaded directly from physionet (https://physionet.org/content/mimiciii/1.4/) once becoming a credentialed user via training course:
* ADMISSIONS
* CHARTEVENTS
* D_ITEMS
* DIAGNOSES_ICD
* ICUSTAYS
* OUTPUTEVENTS
* PATIENTS
* PRESCRIPTIONS
* PROCEDURES_ICD
* SERVICES

The following ER diagram helps illustrate how these tables are linked together: https://pi.cs.oswego.edu/~jmiles3/mimic/assets/MIMIC-ER-DIAGRAM.jpg
  
After downloading the compressed tables from physionet, we unzip the files locally and run a series of preprocessing steps that will ultimately lead to data arrays that we will use to train and evaluate the models. The preprocessing functions are shown in the Preprocessing Code section below. In addition to the MIMIC III tables listed above, the preprocessing functions also generate intermediate data files that can be used in later preprocessing steps that will ultimately lead to constructing the training/testing data arrays.  The implementation for the preprocessing code largely reuses the preprocessing code provided in the GitHub repo of the original paper (https://github.com/sebbarb/time_aware_attention), with some additional comments added and small fixes due to package and python versioning.

All intermediate files that could be compressed to a small enough size to be uploaded to Github are in the data folder of the parent repo.  The original MIMIC III tables which are required for running the preprocessing steps cannot be uploaded to the parent repo, so the preprocessing code shown below represents what works for our team when we have downloaded the files locally.

### Descriptive Statistics

Due to the limitations of the pyhealth library on the MIMIC3Dataset, we were not able to get full descriptive statistics of every table used in this project.  However, for tables that could be parsed using the library, the results are shown in the cells below.  The can run assuming the user downloads the appropriate MIMIC III datasets and uncompresses them locally.

In [85]:
# from pyhealth.datasets import MIMIC3Dataset

# ADMISSIONS and PATIENTS are part of our dataset but don't need to be included in this code since they are parsed by default
# CHARTEVENTS, D_ITEMS, ICUSTAYS, OUTPUTEVENTS, PATIENTS, and SERVICES do not have a parser in this library as indicated
# in the documentation: https://pyhealth.readthedocs.io/en/latest/api/datasets/pyhealth.datasets.MIMIC3Dataset.html

# The code below can be uncommented to generate statistics about a subset of tables used in this project.  The results are pasted in the comment at the bottom.
# dataset = MIMIC3Dataset(
#     root="../MIMIC-III Clinical Database/uncompressed/",
#     tables=["DIAGNOSES_ICD", "PRESCRIPTIONS", "PROCEDURES_ICD"],
# )
# dataset.stat()
# dataset.info()

'''
Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 46520
	- Number of visits: 58976
	- Number of visits per patient: 1.2678
	- Number of events per visit in DIAGNOSES_ICD: 11.0384
	- Number of events per visit in PRESCRIPTIONS: 70.4013
	- Number of events per visit in PROCEDURES_ICD: 4.0711
'''

'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 46520\n\t- Number of visits: 58976\n\t- Number of visits per patient: 1.2678\n\t- Number of events per visit in DIAGNOSES_ICD: 11.0384\n\t- Number of events per visit in PRESCRIPTIONS: 70.4013\n\t- Number of events per visit in PROCEDURES_ICD: 4.0711\n'

### Preprocessing Code

The following 7 preprocessing steps need to be run in the same order as presented in this notebook.  Besides running each cell as such, there are no other requirements or steps in the prepprocessing stage.

In [86]:
# Preprocessing: MIMIC-III ICUSTAYS, PATIENTS, ADMISSIONS, and SERVICES tables to combine into an intermediate dataset that joins these tables together.
mimic_dir = '../MIMIC-III Clinical Database/uncompressed/'
data_dir = './data/'

min_count = 100 # words whose occurred less than min_cnt are encoded as OTHER

def create_icu_pat_admit():
    pd.options.mode.chained_assignment = None  # default='warn'

    # Load icustays table
    # Table purpose: Defines each ICUSTAY_ID in the database, i.e. defines a single ICU stay
    print('Load ICU stays...')
    dtype = {'SUBJECT_ID': 'int32',
           'HADM_ID': 'int32',
           'ICUSTAY_ID': 'int32',
           'INTIME': 'str',
           'OUTTIME': 'str',
           'LOS': 'float32'}
    parse_dates = ['INTIME', 'OUTTIME']
    icustays = pd.read_csv(mimic_dir + 'ICUSTAYS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    
    print('-----------------------------------------')
    
    # Load patients table
    # Table purpose: Contains all charted data for all patients.
    print('Load patients...')
    dtype = {'SUBJECT_ID': 'int32',
           'GENDER': 'str',
           'DOB': 'str',
           'DOD': 'str'}
    parse_dates = ['DOB', 'DOD']
    patients = pd.read_csv(mimic_dir + 'PATIENTS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)  
    
    # Adjust shifted DOBs for older patients (median imputation)
    old_patient = patients['DOB'].dt.year < 2000
    date_offset = pd.DateOffset(years=(300-91), days=(-0.4*365))
    patients['DOB'][old_patient] = patients['DOB'][old_patient].apply(lambda x: x + date_offset)
    
    # Replace GENDER by dummy binary column 
    patients = pd.get_dummies(patients, columns = ['GENDER'], drop_first=True)
    
    print('-----------------------------------------')
    print('Load admissions...')
    # Load admissions table
    # Table purpose: Define a patients hospital admission, HADM_ID.
    dtype = {'SUBJECT_ID': 'int32', 
           'HADM_ID': 'int32',
           'ADMISSION_LOCATION': 'str',
           'INSURANCE': 'str',
           'MARITAL_STATUS': 'str',
           'ETHNICITY': 'str',
           'ADMITTIME': 'str',
           'ADMISSION_TYPE': 'str'}
    parse_dates = ['ADMITTIME']
    admissions = pd.read_csv(mimic_dir + 'ADMISSIONS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    
    print('-----------------------------------------')
    print('Load services...')
    # Load services table
    # Table purpose: Lists services that a patient was admitted/transferred under.
    dtype = {'SUBJECT_ID': 'int32', 
           'HADM_ID': 'int32',
           'TRANSFERTIME': 'str',
           'CURR_SERVICE': 'str'}
    parse_dates = ['TRANSFERTIME']
    services = pd.read_csv(mimic_dir + 'SERVICES.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    
    print('-----------------------------------------')
    
    # Link icustays and patients tables
    print('Link icustays and patients tables...')
    icu_pat = pd.merge(icustays, patients, how='inner', on='SUBJECT_ID')
    icu_pat.sort_values(by=['SUBJECT_ID', 'OUTTIME'], ascending=[True, False], inplace=True)
    assert len(icu_pat['SUBJECT_ID'].unique()) == 46476
    assert len(icu_pat['ICUSTAY_ID'].unique()) == 61532
    
    # Exclude icu stays during which patient died
    icu_pat = icu_pat[~(icu_pat['DOD'] <= icu_pat['OUTTIME'])]
    assert len(icu_pat['SUBJECT_ID'].unique()) == 43126
    assert len(icu_pat['ICUSTAY_ID'].unique()) == 56745
    
    # Determine number of icu discharges in the last 365 days
    print('Compute number of recent admissions...')
    icu_pat['NUM_RECENT_ADMISSIONS'] = 0
    for name, group in tqdm(icu_pat.groupby(['SUBJECT_ID'])):
        for index, row in group.iterrows():
            days_diff = (row['OUTTIME']-group['OUTTIME']).dt.days
            icu_pat.at[index, 'NUM_RECENT_ADMISSIONS'] = len(group[(days_diff > 0) & (days_diff <=365)])
    
    # Create age variable and exclude patients < 18 y.o.
    icu_pat['AGE'] = (icu_pat['OUTTIME'] - icu_pat['DOB']).dt.days/365.
    icu_pat = icu_pat[icu_pat['AGE'] >= 18]
    assert len(icu_pat['SUBJECT_ID'].unique()) == 35233
    assert len(icu_pat['ICUSTAY_ID'].unique()) == 48616
    
    # Time to next admission (discharge to admission!)
    icu_pat['DAYS_TO_NEXT'] = (icu_pat.groupby(['SUBJECT_ID']).shift(1)['INTIME'] - icu_pat['OUTTIME']).dt.days
    
    # Add early readmission flag (less than 30 days after discharge)
    icu_pat['POSITIVE'] = (icu_pat['DAYS_TO_NEXT'] <= 30)
    assert icu_pat['POSITIVE'].sum() == 5495
    
    # Add early death flag (less than 30 days after discharge)
    early_death = ((icu_pat['DOD'] - icu_pat['OUTTIME']).dt.days <= 30)
    assert early_death.sum() == 3795
    
    # Censor negative patients who died within less than 30 days after discharge (no chance of readmission)
    icu_pat = icu_pat[icu_pat['POSITIVE'] | ~early_death]
    assert len(icu_pat['SUBJECT_ID'].unique()) == 33150
    assert len(icu_pat['ICUSTAY_ID'].unique()) == 45298
    
    # Clean up
    icu_pat.drop(columns=['DOB', 'DOD', 'DAYS_TO_NEXT'], inplace=True)
    
    print('-----------------------------------------')
    
    # Link icu_pat and admissions tables
    print('Link icu_pat and admissions tables...')
    icu_pat_admit = pd.merge(icu_pat, admissions, how='left', on=['SUBJECT_ID', 'HADM_ID'])
    print(icu_pat_admit.isnull().sum())
    
    print('Some data cleaning on admissions...')
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('WHITE'), 'ETHNICITY']    = 'WHITE'
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('BLACK'), 'ETHNICITY']    = 'BLACK/AFRICAN AMERICAN'
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('ASIAN'), 'ETHNICITY']    = 'ASIAN'
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('HISPANIC'), 'ETHNICITY'] = 'HISPANIC/LATINO'
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('DECLINED'), 'ETHNICITY'] = 'OTHER/UNKNOWN'
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('MULTI'), 'ETHNICITY']    = 'OTHER/UNKNOWN'
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('UNKNOWN'), 'ETHNICITY']  = 'OTHER/UNKNOWN'
    icu_pat_admit.loc[icu_pat_admit['ETHNICITY'].str.contains('OTHER'), 'ETHNICITY']  = 'OTHER/UNKNOWN'
    
    icu_pat_admit['MARITAL_STATUS'].fillna('UNKNOWN', inplace=True)
    icu_pat_admit.loc[icu_pat_admit['MARITAL_STATUS'].str.contains('MARRIED'), 'MARITAL_STATUS']      = 'MARRIED/LIFE PARTNER'
    icu_pat_admit.loc[icu_pat_admit['MARITAL_STATUS'].str.contains('LIFE PARTNER'), 'MARITAL_STATUS'] = 'MARRIED/LIFE PARTNER'
    icu_pat_admit.loc[icu_pat_admit['MARITAL_STATUS'].str.contains('WIDOWED'), 'MARITAL_STATUS']      = 'WIDOWED/DIVORCED/SEPARATED'
    icu_pat_admit.loc[icu_pat_admit['MARITAL_STATUS'].str.contains('DIVORCED'), 'MARITAL_STATUS']     = 'WIDOWED/DIVORCED/SEPARATED'
    icu_pat_admit.loc[icu_pat_admit['MARITAL_STATUS'].str.contains('SEPARATED'), 'MARITAL_STATUS']    = 'WIDOWED/DIVORCED/SEPARATED'
    icu_pat_admit.loc[icu_pat_admit['MARITAL_STATUS'].str.contains('UNKNOWN'), 'MARITAL_STATUS']      = 'OTHER/UNKNOWN'
    
    columns_to_mask = ['ADMISSION_LOCATION',
                     'INSURANCE',
                     'MARITAL_STATUS',
                     'ETHNICITY']
    icu_pat_admit = icu_pat_admit.apply(lambda x: x.mask(x.map(x.value_counts()) < min_count, 'OTHER/UNKNOWN') if x.name in columns_to_mask else x)                   
    icu_pat_admit = icu_pat_admit.apply(lambda x: x.str.title() if x.name in columns_to_mask else x)
    
    # Compute pre-ICU length of stay in fractional days
    icu_pat_admit['PRE_ICU_LOS'] = (icu_pat_admit['INTIME'] - icu_pat_admit['ADMITTIME']) / np.timedelta64(1, 'D')
    icu_pat_admit.loc[icu_pat_admit['PRE_ICU_LOS']<0, 'PRE_ICU_LOS'] = 0
    
    # Clean up
    icu_pat_admit.drop(columns=['ADMITTIME'], inplace=True)
    
    print('-----------------------------------------')
    
    # Link services table
    # Keep first service only
    services.sort_values(by=['HADM_ID', 'TRANSFERTIME'], ascending=True, inplace=True)
    services = services.groupby(['HADM_ID']).nth(0).reset_index()
    
    # Check if first service is a surgery
    services['SURGERY'] = services['CURR_SERVICE'].str.contains('SURG') | (services['CURR_SERVICE'] == 'ORTHO')
    
    print('Link services table...')  
    icu_pat_admit = pd.merge(icu_pat_admit, services, how='left', on=['SUBJECT_ID', 'HADM_ID'])
    
    # Get elective surgery admissions
    icu_pat_admit['ELECTIVE_SURGERY'] = ((icu_pat_admit['ADMISSION_TYPE'] == 'ELECTIVE') & icu_pat_admit['SURGERY']).astype(int)
    
    # Clean up
    icu_pat_admit.drop(columns=['TRANSFERTIME', 'CURR_SERVICE', 'ADMISSION_TYPE', 'SURGERY'], inplace=True)
    
    print('-----------------------------------------')
    # Baseline characteristics table
    pos = icu_pat_admit[icu_pat_admit['POSITIVE']==1]
    neg = icu_pat_admit[icu_pat_admit['POSITIVE']==0]
    print('Total pos {}'.format(len(pos)))
    print('Total neg {}'.format(len(neg)))
    print(pos['LOS'].describe())
    print(neg['LOS'].describe())
    print((pos['PRE_ICU_LOS']).describe())
    print((neg['PRE_ICU_LOS']).describe())
    pd.set_option('display.precision', 1)
    print(pos['AGE'].describe())
    print(neg['AGE'].describe())
    print(pos['NUM_RECENT_ADMISSIONS'].describe())
    print(neg['NUM_RECENT_ADMISSIONS'].describe())
    print(pd.DataFrame({'COUNTS': pos['GENDER_M'].value_counts(), 'PERC': pos['GENDER_M'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': neg['GENDER_M'].value_counts(), 'PERC': neg['GENDER_M'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': pos['ADMISSION_LOCATION'].value_counts(), 'PERC': pos['ADMISSION_LOCATION'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': neg['ADMISSION_LOCATION'].value_counts(), 'PERC': neg['ADMISSION_LOCATION'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': pos['INSURANCE'].value_counts(), 'PERC': pos['INSURANCE'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': neg['INSURANCE'].value_counts(), 'PERC': neg['INSURANCE'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': pos['MARITAL_STATUS'].value_counts(), 'PERC': pos['MARITAL_STATUS'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': neg['MARITAL_STATUS'].value_counts(), 'PERC': neg['MARITAL_STATUS'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': pos['ETHNICITY'].value_counts(), 'PERC': pos['ETHNICITY'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': neg['ETHNICITY'].value_counts(), 'PERC': neg['ETHNICITY'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': pos['ELECTIVE_SURGERY'].value_counts(), 'PERC': pos['ELECTIVE_SURGERY'].value_counts(normalize=True)*100}))
    print(pd.DataFrame({'COUNTS': neg['ELECTIVE_SURGERY'].value_counts(), 'PERC': neg['ELECTIVE_SURGERY'].value_counts(normalize=True)*100}))  
    print('-----------------------------------------')
    
    print('Save...')
    assert len(icu_pat_admit) == 45298
    icu_pat_admit.sort_values(by='ICUSTAY_ID', ascending=True, inplace=True)
    icu_pat_admit.to_pickle(data_dir + 'icu_pat_admit.pkl')
    icu_pat_admit.to_csv(data_dir + 'icu_pat_admit.csv', index=False)

In [87]:
# Preprocessing: reduce chartevents size by removing implausible measurements, but also add human readable labels to indicate what the chart data represents
# for each entry using data in D_ITEMS
def reduce_chart_events():
    pd.options.mode.chained_assignment = None  # default='warn'
    
    # Relevant ITEMIDs
    gcs_eye_opening          = [184, 220739, 226756, 227011]
    gcs_verbal_response      = [723, 223900, 226758, 227014]
    gcs_motor_response       = [454, 223901, 226757, 227012]
    gcs_total                = [198, 226755]
    diastolic_blood_pressure = [8364, 8368, 8440, 8441, 8502, 8503, 8506, 8555, 220051, 220180, 224643, 225310, 227242]
    systolic_blood_pressure  = [   6,   51,  442,  455, 3313, 3315, 3321, 6701, 220050, 220179, 224167, 225309, 227243]
    mean_blood_pressure      = [52, 443, 456, 2293, 2294, 2647, 3312, 3314, 3320, 6590, 6702, 6927, 7620, 220052, 220181, 225312]
    heart_rate               = [211, 220045, 227018]
    fraction_inspired_oxygen = [189, 190, 727, 1040, 1206, 1863, 2518, 2981, 3420, 3422, 7018, 7041, 7570, 223835, 226754, 227009, 227010]
    respiratory_rate         = [614, 615, 618, 619, 651, 653, 1884, 3603, 6749, 7884, 8113, 220210, 224422, 224688, 224689, 224690, 226774, 227050]
    body_temperature         = [676, 677, 678, 679, 3652, 3654, 6643, 223761, 223762, 226778, 227054]
    weight                   = [763, 3580, 3581, 3582, 3693, 224639, 226512, 226531]
    height                   = [1394, 226707, 226730]
    
    def inch_to_cm(value):
        return value*2.54
    
    def lb_to_kg(value):
        return value/2.205
    
    def oz_to_kg(value):
        return value/35.274
    
    def f_to_c(value):
        return (value-32)*5/9
    
    def frac_to_perc(value):
        return value*100
    
    # Relevant ITEMIDs
    body_temperature_F       = [678, 679, 3652, 3654, 6643, 223761, 226778, 227054]
    weight_lb                = [3581, 226531]
    weight_oz                = [3582]
    height_inch              = [1394, 226707]
    
    relevant_ids = (gcs_eye_opening + gcs_verbal_response + gcs_motor_response + gcs_total + mean_blood_pressure + 
                  heart_rate + fraction_inspired_oxygen + respiratory_rate + body_temperature + weight + height)
    
    print('-----------------------------------------')
    print('Load item definitions')
    dtype = {'ITEMID': 'int32',
           'LABEL': 'str',
           'UNITNAME': 'str'}
    defs = pd.read_csv(mimic_dir + 'D_ITEMS.csv', usecols=dtype.keys(), dtype=dtype)
    print('GCS_EYE_OPENING')
    print(defs[defs['ITEMID'].isin(gcs_eye_opening)])
    print('GCS_VERBAL_RESPONSE')
    print(defs[defs['ITEMID'].isin(gcs_verbal_response)])
    print('GCS_MOTOR_RESPONSE')
    print(defs[defs['ITEMID'].isin(gcs_motor_response)])
    print('GCS_TOTAL')
    print(defs[defs['ITEMID'].isin(gcs_total)])
    print('DIASTOLIC_BP')
    print(defs[defs['ITEMID'].isin(diastolic_blood_pressure)])
    print('SYSTOLIC_BP')
    print(defs[defs['ITEMID'].isin(systolic_blood_pressure)])
    print('MEAN_BP')
    print(defs[defs['ITEMID'].isin(mean_blood_pressure)])
    print('HEART_RATE')
    print(defs[defs['ITEMID'].isin(heart_rate)])
    print('FRACTION_INSPIRED_OXYGEN')
    print(defs[defs['ITEMID'].isin(fraction_inspired_oxygen)])
    print('RESPIRATORY_RATE')
    print(defs[defs['ITEMID'].isin(respiratory_rate)])
    print('BODY_TEMPERATURE')
    print(defs[defs['ITEMID'].isin(body_temperature)])
    print('WEIGHT')
    print(defs[defs['ITEMID'].isin(weight)])
    print('HEIGHT')
    print(defs[defs['ITEMID'].isin(height)])
    print('-----------------------------------------')
    
    print('Loading Chart Events')
    dtype = {'SUBJECT_ID': 'int32',
           'HADM_ID': 'int32',
           'ICUSTAY_ID': 'str',
           'ITEMID': 'int32',
           'CHARTTIME': 'str',
           'VALUENUM': 'float32'}
    parse_dates = ['CHARTTIME']
    # Load chartevents table
    # Table purpose: Contains all charted data for all patients.
    chunksize = 1000000
    i = 0
    # Not parsing dates
    for df in tqdm(pd.read_csv(mimic_dir + 'CHARTEVENTS.csv', usecols=dtype.keys(), dtype=dtype, chunksize=chunksize)):
        df = df[df['ICUSTAY_ID'].notna() & df['VALUENUM'].notna() & (df['ITEMID'].isin(relevant_ids)) & (df['VALUENUM'] > 0)]
        # convert units
        df.loc[df['ITEMID'].isin(body_temperature_F), 'VALUENUM'] = f_to_c(df[df['ITEMID'].isin(body_temperature_F)].VALUENUM)
        df.loc[df['ITEMID'].isin(weight_lb), 'VALUENUM'] = lb_to_kg(df[df['ITEMID'].isin(weight_lb)].VALUENUM)
        df.loc[df['ITEMID'].isin(weight_oz), 'VALUENUM'] = oz_to_kg(df[df['ITEMID'].isin(weight_oz)].VALUENUM)
        df.loc[df['ITEMID'].isin(height_inch), 'VALUENUM'] = inch_to_cm(df[df['ITEMID'].isin(height_inch)].VALUENUM)
        df.loc[(df['ITEMID'].isin(fraction_inspired_oxygen)) & (df['VALUENUM']<=1), 'VALUENUM'] = frac_to_perc(df[(df['ITEMID'].isin(fraction_inspired_oxygen)) & (df['VALUENUM']<=1)].VALUENUM)
        # remove implausible measurements
        df = df[~(df['ITEMID'].isin(gcs_total) & (df.VALUENUM < 3))]
        df = df[~(df['ITEMID'].isin(diastolic_blood_pressure + systolic_blood_pressure + mean_blood_pressure) & (df.VALUENUM > 250))]
        df = df[~(df['ITEMID'].isin(heart_rate) & ((df.VALUENUM < 1) | (df.VALUENUM > 250)))]
        df = df[~(df['ITEMID'].isin(fraction_inspired_oxygen) & (df.VALUENUM > 100))]
        df = df[~(df['ITEMID'].isin(respiratory_rate) & ((df.VALUENUM < 1) | (df.VALUENUM > 100)))]
        df = df[~(df['ITEMID'].isin(body_temperature) & (df.VALUENUM > 50))]
        df = df[~(df['ITEMID'].isin(weight) & (df.VALUENUM > 700))]
        df = df[~(df['ITEMID'].isin(height) & (df.VALUENUM > 300))]
        df = df[df['VALUENUM'] > 0]
        # label
        df['CE_TYPE'] = ''
        df.loc[df['ITEMID'].isin(gcs_eye_opening), 'CE_TYPE'] = 'GCS_EYE_OPENING'
        df.loc[df['ITEMID'].isin(gcs_verbal_response), 'CE_TYPE'] = 'GCS_VERBAL_RESPONSE'
        df.loc[df['ITEMID'].isin(gcs_motor_response), 'CE_TYPE'] = 'GCS_MOTOR_RESPONSE'
        df.loc[df['ITEMID'].isin(gcs_total), 'CE_TYPE'] = 'GCS_TOTAL'
        df.loc[df['ITEMID'].isin(diastolic_blood_pressure), 'CE_TYPE'] = 'DIASTOLIC_BP'
        df.loc[df['ITEMID'].isin(systolic_blood_pressure), 'CE_TYPE'] = 'SYSTOLIC_BP'
        df.loc[df['ITEMID'].isin(mean_blood_pressure), 'CE_TYPE'] = 'MEAN_BP'
        df.loc[df['ITEMID'].isin(heart_rate), 'CE_TYPE'] = 'HEART_RATE'
        df.loc[df['ITEMID'].isin(fraction_inspired_oxygen), 'CE_TYPE'] = 'FRACTION_INSPIRED_OXYGEN'
        df.loc[df['ITEMID'].isin(respiratory_rate), 'CE_TYPE'] = 'RESPIRATORY_RATE'
        df.loc[df['ITEMID'].isin(body_temperature), 'CE_TYPE'] = 'BODY_TEMPERATURE'
        df.loc[df['ITEMID'].isin(weight), 'CE_TYPE'] = 'WEIGHT'
        df.loc[df['ITEMID'].isin(height), 'CE_TYPE'] = 'HEIGHT'    
        df.drop(columns=['ITEMID'], inplace=True)
        
        # save
        if i == 0:
            df.to_csv(data_dir + 'chartevents_reduced.csv', index=False)
        else:
            df.to_csv(data_dir + 'chartevents_reduced.csv', mode='a', header=False, index=False)
        i += 1

In [88]:
# Preprocessing: reduce outputs events step by filtering on events specific to urine output, determined by data in D_ITEMS
def reduce_output_events():
    pd.options.mode.chained_assignment = None  # default='warn'
    
    # Relevant ITEMIDs, from https://github.com/vincentmajor/mimicfilters/blob/master/lists/OASIS_components/preprocess_urine_awk_str.txt
    urine_output = [42810, 43171, 43173, 43175, 43348, 43355, 43365, 43372, 43373, 43374, 43379, 43380, 43431, 43462, 43522, 40405, 40428, 40534, 
    40288, 42042, 42068, 42111, 42119, 42209, 41857, 40715, 40056, 40061, 40085, 40094, 40096, 42001, 42676, 42556, 43093, 44325, 44706,
    44506, 42859, 44237, 44313, 44752, 44824, 44837, 43576, 43589, 43633, 44911, 44925, 42362, 42463, 42507, 42510, 40055, 40057, 40065,
    40069, 45804, 45841, 43811, 43812, 43856, 43897, 43931, 43966, 44080, 44103, 44132, 45304, 46177, 46532, 46578, 46658, 46748, 40651,
    43053, 43057, 40473, 42130, 41922, 44253, 44278, 46180, 44684, 43333, 43347, 42592, 42666, 42765, 42892, 45927, 44834, 43638, 43654,
    43519, 43537, 42366, 45991, 46727, 46804, 43987, 44051, 227489, 226566, 226627, 226631, 45415, 42111, 41510, 40055, 226559, 40428,
    40580, 40612, 40094, 40848, 43685, 42362, 42463, 42510, 46748, 40972, 40973, 46456, 226561, 226567, 226632, 40096, 40651, 226557,
    226558, 40715, 226563]
    
    # Relevant ITEMIDs
    print('-----------------------------------------')
    print('Load item definitions')
    dtype = {'ITEMID': 'int32',
           'LABEL': 'str',
           'UNITNAME': 'str',
           'LINKSTO': 'str'}
    defs = pd.read_csv(mimic_dir + 'D_ITEMS.csv', usecols=dtype.keys(), dtype=dtype)
    print('URINE_OUTPUT')
    defs = defs[defs['ITEMID'].isin(urine_output)]
    defs['LABEL'] = defs['LABEL'].str.lower()
    # Remove measurements in /kg/hr
    defs = defs[~(defs['LABEL'].str.contains('hr') | defs['LABEL'].str.contains('kg')) | defs['LABEL'].str.contains('nephro')]
    print(defs['LABEL'])
    urine_output = defs['ITEMID'].tolist()
    print('-----------------------------------------')
    
    print('Loading Output Events')
    dtype = {'ICUSTAY_ID': 'str',
           'ITEMID': 'int32',
           'CHARTTIME': 'str',
           'VALUE': 'float32'}
    parse_dates = ['CHARTTIME']
    
    # Load outputevents table
    # Table purpose: Output data for patients.
    df = pd.read_csv(mimic_dir + 'OUTPUTEVENTS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    df = df.rename(columns={'VALUE': 'VALUENUM'})
    df = df[df['ICUSTAY_ID'].notna() & df['VALUENUM'].notna() & (df['ITEMID'].isin(urine_output)) & (df['VALUENUM'] > 0)]
    df['ICUSTAY_ID'] = df['ICUSTAY_ID'].astype('int32')
    
    # remove implausible measurements
    df = df[~(df.VALUENUM > 10000)]
    
    # sum all outputs in one day
    df.drop(columns=['ITEMID'], inplace=True)
    df['CHARTTIME'] = df['CHARTTIME'].dt.date
    df = df.groupby(['ICUSTAY_ID', 'CHARTTIME']).sum()
    df['CE_TYPE'] = 'URINE_OUTPUT'
    df = df[~(df.VALUENUM > 10000)]
    
    print('Remove admission and discharge days (since data on urine output is incomplete)')
    # Load icustays table
    # Table purpose: Defines each ICUSTAY_ID in the database, i.e. defines a single ICU stay
    print('Load ICU stays...')
    dtype = {'ICUSTAY_ID': 'int32',
           'INTIME': 'str',
           'OUTTIME': 'str'}
    parse_dates = ['INTIME', 'OUTTIME']
    icustays = pd.read_csv(mimic_dir + 'ICUSTAYS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    icustays['INTIME'] = icustays['INTIME'].dt.date
    icustays['OUTTIME'] = icustays['OUTTIME'].dt.date
    
    # Merge
    tmp = icustays[['ICUSTAY_ID', 'INTIME']].drop_duplicates()
    tmp = tmp.rename(columns={'INTIME': 'CHARTTIME'})
    tmp['ID_IN'] = 1
    df = pd.merge(df, tmp, how='left', on=['ICUSTAY_ID', 'CHARTTIME'])
    tmp = icustays[['ICUSTAY_ID', 'OUTTIME']].drop_duplicates()
    tmp = tmp.rename(columns={'OUTTIME': 'CHARTTIME'})
    tmp['ID_OUT'] = 1
    df = pd.merge(df, tmp, how='left', on=['ICUSTAY_ID', 'CHARTTIME'])
    
    # Remove admission and discharge days
    df = df[df['ID_IN'].isnull() & df['ID_OUT'].isnull()]
    df.drop(columns=['ID_IN', 'ID_OUT'], inplace=True)
    
    # Add SUBJECT_ID and HADM_ID
    icustays.drop(columns=['INTIME', 'OUTTIME'], inplace=True)  
    df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME']) + pd.DateOffset(hours=12)
    
    # Save
    df.to_pickle(data_dir + 'outputevents_reduced.pkl')

In [89]:
# Preprocessing: Create a merged intermediate output file which contains content from the reduced chartevents and reduced output events generated
# in the previous two steps
def merge_chart_outputs():
    pd.options.mode.chained_assignment = None  # default='warn'

    # Load (reduced) chartevents table
    print('Loading chart events...')
    dtype = {'SUBJECT_ID': 'int32',
           'ICUSTAY_ID': 'int32',
           'CE_TYPE': 'str',
           'CHARTTIME': 'str',
           'VALUENUM': 'float32'}
    parse_dates = ['CHARTTIME']
    charts = pd.read_csv(data_dir + 'chartevents_reduced.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    
    print('-----------------------------------------')
    
    print('Compute BMI and GCS total...')
    charts.sort_values(by=['SUBJECT_ID', 'ICUSTAY_ID', 'CHARTTIME'], ascending=[True, True, False], inplace=True)
    
    # Compute BMI
    rows_bmi = (charts['CE_TYPE']=='WEIGHT') | (charts['CE_TYPE']=='HEIGHT')
    charts_bmi = charts[rows_bmi]
    charts_bmi = charts_bmi.pivot_table(index=['SUBJECT_ID', 'ICUSTAY_ID', 'CHARTTIME'], columns='CE_TYPE', values='VALUENUM')
    charts_bmi = charts_bmi.rename_axis(None, axis=1).reset_index()
    charts_bmi['HEIGHT'] = charts_bmi.groupby('SUBJECT_ID')['HEIGHT'].ffill()
    charts_bmi['HEIGHT'] = charts_bmi.groupby('SUBJECT_ID')['HEIGHT'].bfill()
    charts_bmi =  charts_bmi[~pd.isnull(charts_bmi).any(axis=1)]
    charts_bmi['VALUENUM'] = charts_bmi['WEIGHT']/charts_bmi['HEIGHT']/charts_bmi['HEIGHT']*10000
    charts_bmi['CE_TYPE'] = 'BMI'
    charts_bmi.drop(columns=['HEIGHT', 'WEIGHT'], inplace=True)
    
    # Compute GCS total if not available
    rows_gcs = (charts['CE_TYPE']=='GCS_EYE_OPENING') | (charts['CE_TYPE']=='GCS_VERBAL_RESPONSE') | (charts['CE_TYPE']=='GCS_MOTOR_RESPONSE') | (charts['CE_TYPE']=='GCS_TOTAL')
    charts_gcs = charts[rows_gcs]
    charts_gcs = charts_gcs.pivot_table(index=['SUBJECT_ID', 'ICUSTAY_ID', 'CHARTTIME'], columns='CE_TYPE', values='VALUENUM')
    charts_gcs = charts_gcs.rename_axis(None, axis=1).reset_index()
    null_gcs_total = charts_gcs['GCS_TOTAL'].isnull()
    charts_gcs.loc[null_gcs_total, 'GCS_TOTAL'] = charts_gcs[null_gcs_total].GCS_EYE_OPENING + charts_gcs[null_gcs_total].GCS_VERBAL_RESPONSE + charts_gcs[null_gcs_total].GCS_MOTOR_RESPONSE
    charts_gcs =  charts_gcs[~charts_gcs['GCS_TOTAL'].isnull()]
    charts_gcs = charts_gcs.rename(columns={'GCS_TOTAL': 'VALUENUM'})
    charts_gcs['CE_TYPE'] = 'GCS_TOTAL'
    charts_gcs.drop(columns=['GCS_EYE_OPENING', 'GCS_VERBAL_RESPONSE', 'GCS_MOTOR_RESPONSE'], inplace=True)
    
    # Merge back with rest of the table
    rows_others = ~rows_bmi & ~rows_gcs
    charts = pd.concat([charts_bmi, charts_gcs, charts[rows_others]], ignore_index=True, sort=False)
    charts.drop(columns=['SUBJECT_ID'], inplace=True)
    charts.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], ascending=[True, False], inplace=True)
    
    print('-----------------------------------------')
    
    # Load (reduced) outputevents table
    print('Loading output events...')
    outputs = pd.read_pickle(data_dir + 'outputevents_reduced.pkl')
    df = pd.concat([charts, outputs], ignore_index=True, sort=False)
    df.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], ascending=[True, False], inplace=True)
    
    print('-----------------------------------------')
    
    print('Create categorical variable...')
    # Bin according to OASIS severity score
    heart_rate_bins               = np.array([-1, 32.99, 88.5, 106.5, 125.5, np.Inf])
    respiratory_rate_bins         = np.array([-1, 5.99, 12.5, 22.5, 30.5, 44.5, np.Inf])
    body_temperature_bins         = np.array([-1, 33.21, 35.93, 36.39, 36.88, 39.88, np.Inf])
    mean_bp_bins                  = np.array([-1, 20.64, 50.99, 61.32, 143.44, np.Inf])
    fraction_inspired_oxygen_bins = np.array([-1, np.Inf])
    gcs_total_bins                = np.array([-1, 7, 13, 14, 15])
    bmi_bins                      = np.array([-1, 15, 16, 18.5, 25, 30, 35, 40, 45, 50, 60, np.Inf])
    urine_output_bins             = np.array([-1, 670.99, 1426.99, 2543.99, 6896, np.Inf])
    bins = [heart_rate_bins, respiratory_rate_bins, body_temperature_bins, mean_bp_bins, fraction_inspired_oxygen_bins, gcs_total_bins, urine_output_bins]

    # Labels 
    heart_rate_labels               = ['CHART_HR_m1', 'CHART_HR_n', 'CHART_HR_p1', 'CHART_HR_p2', 'CHART_HR_p3']
    respiratory_rate_labels         = ['CHART_RR_m2', 'CHART_RR_m1', 'CHART_RR_n', 'CHART_RR_p1', 'CHART_RR_p2', 'CHART_RR_p3']
    body_temperature_labels         = ['CHART_BT_m3', 'CHART_BT_m2', 'CHART_BT_m1', 'CHART_BT_n', 'CHART_BT_p1', 'CHART_BT_p2']
    mean_bp_labels                  = ['CHART_BP_m3', 'CHART_BP_m2', 'CHART_BP_m1', 'CHART_BP_n', 'CHART_BP_p1']
    fraction_inspired_oxygen_labels = ['CHART_VENT']
    gcs_total_labels                = ['CHART_GC_m3', 'CHART_GC_m2', 'CHART_GC_m1', 'CHART_GC_n']
    bmi_labels                      = ['CHART_BM_m3', 'CHART_BM_m2', 'CHART_BM_m1', 'CHART_BM_n', 'CHART_BM_p1', 'CHART_BM_p2', 'CHART_BM_p3', 'CHART_BM_p4', 'CHART_BM_p5', 'CHART_BM_p6', 'CHART_BM_p7']
    urine_output_labels             = ['CHART_UO_m3', 'CHART_UO_m2', 'CHART_UO_m1', 'CHART_UO_n', 'CHART_UO_p1']
    labels = [heart_rate_labels, respiratory_rate_labels, body_temperature_labels, mean_bp_labels, fraction_inspired_oxygen_labels, gcs_total_labels, urine_output_labels]

    # Chart event types
    ce_types = ['HEART_RATE', 'RESPIRATORY_RATE', 'BODY_TEMPERATURE', 'MEAN_BP', 'FRACTION_INSPIRED_OXYGEN', 'GCS_TOTAL', 'URINE_OUTPUT']
    
    df_list = []
    df_list_last_only = [] # for logistic regression
    for type, label, bin in zip(ce_types, labels, bins):
        # get chart events of a specific type
        tmp = df[df['CE_TYPE'] == type]

        # bin them and sort
        tmp['VALUECAT'] = pd.cut(tmp['VALUENUM'], bins=bin, labels=label)
        tmp.drop(columns=['CE_TYPE', 'VALUENUM'], inplace=True)
        tmp.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], ascending=[True, False], inplace=True)

        # remove consecutive duplicates
        tmp = tmp[(tmp[['ICUSTAY_ID', 'VALUECAT']] != tmp[['ICUSTAY_ID', 'VALUECAT']].shift()).any(axis=1)]
        df_list.append(tmp)

        # for logistic regression, keep only the last measurement
        tmp = tmp.drop_duplicates(subset='ICUSTAY_ID')
        df_list_last_only.append(tmp)
    
    df = pd.concat(df_list, ignore_index=True, sort=False)
    df.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], ascending=[True, False], inplace=True)
    
    # drop duplicates to keep size manageable
    df = df.drop_duplicates()
    
    print('-----------------------------------------')
    
    print('Save...')
    df.to_pickle(data_dir + 'charts_outputs_reduced.pkl')
    df.to_csv(data_dir + 'charts_outputs_reduced.csv', index=False)
    
    print('-----------------------------------------')
    
    print('Save data for logistic regression...')
    
    # for logistic regression
    df_last_only = pd.concat(df_list_last_only, ignore_index=True, sort=False)
    df_last_only.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], ascending=[True, False], inplace=True)
    df_last_only.to_pickle(data_dir + 'charts_outputs_last_only.pkl')
    df_last_only.to_csv(data_dir + 'charts_outputs_last_only.csv', index=False)

In [90]:
# Preprocessing: Links diagnoses with their associated procedures
def link_diagnoses_procedures():
    pd.options.mode.chained_assignment = None  # default='warn'

    # Load icu_pat table
    print('Loading icu_pat...')
    icu_pat = pd.read_pickle(data_dir + 'icu_pat_admit.pkl')
    
    print('-----------------------------------------')
    print('Load admissions...')
    # Load admissions table
    # Table purpose: Define a patients hospital admission, HADM_ID.
    dtype = {'HADM_ID': 'int32',
           'ADMITTIME': 'str',
           'DISCHTIME': 'str'}
    parse_dates = ['ADMITTIME', 'DISCHTIME']
    admissions = pd.read_csv(mimic_dir + 'ADMISSIONS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    
    print('-----------------------------------------')
    print('Load diagnoses and procedures...')
    dtype = {'SUBJECT_ID': 'int32',
           'HADM_ID': 'int32',
           'ICD9_CODE': 'str'}

    # Load diagnosis_icd table
    # Table purpose: Contains ICD diagnoses for patients, most notably ICD-9 diagnoses.
    diagnoses = pd.read_csv(mimic_dir + 'DIAGNOSES_ICD.csv', usecols=dtype.keys(), dtype=dtype)
    diagnoses = diagnoses.dropna()

    # Load procedures_icd table
    # Table purpose: Contains ICD procedures for patients, most notably ICD-9 procedures.
    procedures = pd.read_csv(mimic_dir + 'PROCEDURES_ICD.csv', usecols=dtype.keys(), dtype=dtype)
    procedures = procedures.dropna()
    
    # Merge diagnoses and procedures
    diagnoses['ICD9_CODE'] = 'DIAGN_' + diagnoses['ICD9_CODE'].str.lower().str.strip()
    procedures['ICD9_CODE'] = 'PROCE_' + procedures['ICD9_CODE'].str.lower().str.strip()
    diag_proc = pd.concat([diagnoses, procedures], ignore_index=True, sort=False)
    
    print('-----------------------------------------')
    
    # Link diagnoses/procedures and admissions tables
    print('Link diagnoses/procedures and admissions tables...')
    diag_proc = pd.merge(diag_proc, admissions, how='inner', on='HADM_ID').drop(columns=['HADM_ID'])
    
    # Link diagnoses/procedures and icu_pat tables
    print('Link diagnoses/procedures and icu_pat tables...')
    diag_proc = pd.merge(icu_pat[['SUBJECT_ID', 'ICUSTAY_ID', 'OUTTIME']], diag_proc, how='left', on=['SUBJECT_ID'])
    
    # Remove codes related to future admissions using time difference to ADMITTIME
    diag_proc['DAYS_TO_OUT'] = (diag_proc['OUTTIME']-diag_proc['ADMITTIME']) / np.timedelta64(1, 'D')
    diag_proc = diag_proc[(diag_proc['DAYS_TO_OUT'] >= 0) | diag_proc['DAYS_TO_OUT'].isna()]

    # Reset time value using time difference to DISCHTIME (0 if negative)
    diag_proc['DAYS_TO_OUT'] = (diag_proc['OUTTIME']-diag_proc['DISCHTIME']) / np.timedelta64(1, 'D')
    diag_proc.loc[diag_proc['DAYS_TO_OUT'] < 0, 'DAYS_TO_OUT'] = 0
    diag_proc = diag_proc.drop(columns=['SUBJECT_ID', 'OUTTIME', 'ADMITTIME', 'DISCHTIME'])

    # Lost some ICUSTAY_IDs with only negative DAYS_TO_OUT, merge back
    diag_proc = pd.merge(icu_pat[['ICUSTAY_ID']], diag_proc, how='left', on=['ICUSTAY_ID'])
    
    print('Drop duplicates...')
    diag_proc = diag_proc.drop_duplicates()
    
    print('Map rare codes to OTHER...')
    diag_proc = diag_proc.apply(lambda x: x.mask(x.map(x.value_counts()) < min_count, 'other') if x.name in ['ICD9_CODE'] else x)                   
    
    print('-----------------------------------------')  
    print('Save...')
    assert len(diag_proc['ICUSTAY_ID'].unique()) == 45298
    diag_proc.sort_values(by=['ICUSTAY_ID', 'DAYS_TO_OUT'], ascending=[True, True], inplace=True)
    diag_proc.to_pickle(data_dir + 'diag_proc.pkl')
    diag_proc.to_csv(data_dir + 'diag_proc.csv', index=False)

In [91]:
# Preprocessing: Links chartevents and outputevents data with relevant prescriptions
def link_charts_prescriptions():
    pd.options.mode.chained_assignment = None  # default='warn'

    # Load icu_pat table
    print('Loading icu_pat...')
    icu_pat = pd.read_pickle(data_dir + 'icu_pat_admit.pkl')
    
    print('-----------------------------------------')
    print('Load charts and outputs...')
    charts_outputs = pd.read_pickle(data_dir + 'charts_outputs_reduced.pkl')
    
    print('-----------------------------------------')
    print('Load prescriptions...')
    dtype = {'ICUSTAY_ID': 'str',
           'DRUG': 'str',
           'STARTDATE': 'str'}
    parse_dates = ['STARTDATE']
    # Load prescriptions table
    # Table purpose: Contains medication related order entries, i.e. prescriptions
    prescriptions = pd.read_csv(mimic_dir + 'PRESCRIPTIONS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
    prescriptions = prescriptions.dropna()
    prescriptions['ICUSTAY_ID'] = prescriptions['ICUSTAY_ID'].astype('int32')
    prescriptions['DRUG'] = 'PRESC_' + prescriptions['DRUG'].str.lower().replace('\s+', '', regex=True)
    prescriptions = prescriptions.rename(columns={'DRUG': 'VALUECAT', 'STARTDATE': 'CHARTTIME'})
    df = pd.concat([charts_outputs, prescriptions], ignore_index=True, sort=False)
    
    print('-----------------------------------------')
    
    # Link charts/outputs and icu_pat tables
    print('Link charts/outputs and icu_pat tables...')
    df = pd.merge(icu_pat[['ICUSTAY_ID', 'OUTTIME']], df, how='left', on=['ICUSTAY_ID'])
    
    # Reset time value using time difference to DISCHTIME (0 if negative)
    df['HOURS_TO_OUT'] = (df['OUTTIME']-df['CHARTTIME']) / np.timedelta64(1, 'h')
    df.loc[df['HOURS_TO_OUT'] < 0, 'HOURS_TO_OUT'] = 0
    df = df.drop(columns=['OUTTIME', 'CHARTTIME'])
    
    print('Drop duplicates...')
    df = df.drop_duplicates()
    
    print('Map rare codes to OTHER...')
    df = df.apply(lambda x: x.mask(x.map(x.value_counts()) < min_count, 'other') if x.name in ['VALUECAT'] else x)                   
    
    print('-----------------------------------------')  
    print('Save...')
    assert len(df['ICUSTAY_ID'].unique()) == 45298
    df.sort_values(by=['ICUSTAY_ID', 'HOURS_TO_OUT'], ascending=[True, True], inplace=True)
    df.to_pickle(data_dir + 'charts_prescriptions.pkl')
    df.to_csv(data_dir + 'charts_prescriptions.csv', index=False)

In [92]:
# Preprocessing: creates data arrays step used for training/validating/testing all models
def create_data_arrays():
    pd.options.mode.chained_assignment = None  # default='warn'

    def get_arrays(df, code_column, time_column, quantile=1):
      df['COUNT'] = df.groupby(['ICUSTAY_ID']).cumcount()
      df = df[df['COUNT'] < df.groupby(['ICUSTAY_ID']).size().quantile(q=quantile)]
      max_count_df = df['COUNT'].max()+1
      print('max_count {}'.format(max_count_df))
      multiindex_df = pd.MultiIndex.from_product([icu_pat['ICUSTAY_ID'], range(max_count_df)], names = ['ICUSTAY_ID', 'COUNT'])
      df = df.set_index(['ICUSTAY_ID', 'COUNT'])
    
      print('Reindex df...')
      df = df.reindex(multiindex_df).fillna(0)
      print('done')
      df_times = df[time_column].values.reshape((num_icu_stays, max_count_df))
      df[code_column] = df[code_column].astype('category')
      dict_df = dict(enumerate(df[code_column].cat.categories))
      df[code_column] = df[code_column].cat.codes
      df = df[code_column].values.reshape((num_icu_stays, max_count_df))
    
      return df, df_times, dict_df
  
    # Load icu_pat table
    print('Loading icu_pat...')
    icu_pat = pd.read_pickle(data_dir + 'icu_pat_admit.pkl')
    
    print('Loading diagnoses/procedures...')
    dp = pd.read_pickle(data_dir + 'diag_proc.pkl')
    
    print('Loading charts/prescriptions...')
    cp = pd.read_pickle(data_dir + 'charts_prescriptions.pkl')
    
    print('-----------------------------------------')
    
    num_icu_stays = len(icu_pat['ICUSTAY_ID'])
    
    # static variables
    print('Create static array...')
    icu_pat = pd.get_dummies(icu_pat, columns = ['ADMISSION_LOCATION', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY'])
    icu_pat.drop(columns=['ADMISSION_LOCATION_Emergency Room Admit', 'INSURANCE_Medicare', 'MARITAL_STATUS_Married/Life Partner', 'ETHNICITY_White'], inplace=True) # drop reference columns
    static_columns = icu_pat.columns.str.contains('AGE|GENDER_M|LOS|NUM_RECENT_ADMISSIONS|ADMISSION_LOCATION|INSURANCE|MARITAL_STATUS|ETHNICITY|PRE_ICU_LOS|ELECTIVE_SURGERY')
    static = icu_pat.loc[:, static_columns].values
    static_vars = icu_pat.loc[:, static_columns].columns.values.tolist()
    
    # classification label
    print('Create label array...')
    label = icu_pat.loc[:, 'POSITIVE'].values
    
    # diagnoses/procedures and charts/prescriptions
    print('Create diagnoses/procedures and charts/prescriptions array...')
    dp, dp_times, dict_dp = get_arrays(dp, 'ICD9_CODE', 'DAYS_TO_OUT', 1)
    cp, cp_times, dict_cp = get_arrays(cp, 'VALUECAT', 'HOURS_TO_OUT', 0.95)
    
    # Normalize times
    dp_times = dp_times/dp_times.max()
    cp_times = cp_times/cp_times.max()
    
    print('-----------------------------------------')
    
    print('Split data into train/validate/test...')
    # Split patients to avoid data leaks
    patients = icu_pat['SUBJECT_ID'].drop_duplicates()
    train, validate, test = np.split(patients.sample(frac=1, random_state=123), [int(.9*len(patients)), int(.9*len(patients))])
    train_ids = icu_pat['SUBJECT_ID'].isin(train).values
    validate_ids = icu_pat['SUBJECT_ID'].isin(validate).values
    test_ids = icu_pat['SUBJECT_ID'].isin(test).values
    
    print('Get patients corresponding to test ids')
    test_ids_patients = icu_pat['SUBJECT_ID'].iloc[test_ids].reset_index(drop=True)
    
    print('-----------------------------------------')
    
    print('Save...')
    np.savez(data_dir + 'data_arrays.npz', static=static, static_vars=static_vars, label=label,
           dp=dp, cp=cp, dp_times=dp_times, cp_times=cp_times, dict_dp=dict_dp, dict_cp=dict_cp,
           train_ids=train_ids, validate_ids=validate_ids, test_ids=test_ids)
    # np.savez(data_dir + 'data_dictionaries.npz', dict_dp=dict_dp, dict_cp=dict_cp)
    test_ids_patients.to_pickle(data_dir + 'test_ids_patients.pkl')

In [93]:
# Order in which the preprocessing steps must be run in order to produce the final data_arrays.npz and test_ids_patients.pkl files needed for training/validating/testing

# create_icu_pat_admit()
# reduce_chart_events()
# reduce_output_events()
# merge_chart_outputs()
# link_diagnoses_procedures()
# link_charts_prescriptions()
# create_data_arrays()

In [94]:
  batch_size = 128
  num_epochs = 80
  dropout_rate = 0.5
  patience = 10 # early stopping
  

In [95]:
def get_data(data, type):
  # Data
  static       = data['static'].astype('float32')
  label        = data['label'].astype('float32')
  dp           = data['dp'].astype('int64') # diagnoses/procedures
  cp           = data['cp'].astype('int64') # charts/prescriptions
  dp_times     = data['dp_times'].astype('float32')
  cp_times     = data['cp_times'].astype('float32')
  train_ids    = data['train_ids']
  validate_ids = data['validate_ids']
  test_ids     = data['test_ids']  

  if (type == 'TRAIN'):
    ids = train_ids
  elif (type == 'VALIDATE'):
    ids = validate_ids
  elif (type == 'TEST'):
    ids = test_ids
  elif (type == 'ALL'):
    ids = np.full_like(label, True, dtype=bool)

  static   = static[ids, :]
  label    = label[ids]
  dp       = dp[ids, :]
  cp       = cp[ids, :]
  dp_times = dp_times[ids, :]
  cp_times = cp_times[ids, :]
  
  return static, dp, cp, dp_times, cp_times, label

In [96]:


def get_trainloader(data, type, shuffle=True, idx=None):
  # Data
  static, dp, cp, dp_times, cp_times, label = get_data(data, type)

  # Bootstrap
  if idx is not None:
    static, dp, cp, dp_times, cp_times, label = static[idx], dp[idx], cp[idx], dp_times[idx], cp_times[idx], label[idx]

  # Compute total batch count
  num_batches = len(label) // batch_size
  
  # Create dataset
  dataset = utils.TensorDataset(torch.from_numpy(static), 
                                torch.from_numpy(dp),
                                torch.from_numpy(cp),
                                torch.from_numpy(dp_times),
                                torch.from_numpy(cp_times),
                                torch.from_numpy(label))

  # # Create batch queues
  trainloader = utils.DataLoader(dataset,
                                 batch_size = batch_size, 
                                 shuffle = shuffle,
                                 sampler = None,
                                 num_workers = 2,
                                 drop_last = True)
                                 
  # # Weight of positive samples for training
  pos_weight = torch.tensor((len(label) - np.sum(label))/np.sum(label))
  
  return trainloader, num_batches, pos_weight
  

##   Model

The combination of the different approaches that we plan to experiment and evaluate the outcomes are listed below:
  * RNN (concatenated Δtime)
  * RNN (concatenated Δtime) + Attention
  * Attention (concatenated time)
  * ODE + RNN + Attention
  * ODE + Attention
  * ODE + RNN    
  * RNN (exp time decay) + Attention
  * RNN (exp time decay)    
  * RNN (ODE time decay)   
  * RNN (ODE time decay) + Attention    
  * MCE + RNN + Attention
  * MCE + RNN
  * MCE + Attention
  * Logistic Regression

All the above models have been implemented - trained and tested.
Each model is defined in its own class, with the comments in the class outlining each step. 

Training is done on the output of the preprocessing step - data_arrays.npz.

Loss function : BCEWithLogitsLoss

Optimizer : Adam for stochastic gradient descent

The trained model definitions are stored under "logdir". A separate folder by model captures the "final_model.pt" file.
Model need not be trained every time, it does take on an average of 6-8 hours for training due to the data volumes and the devidce being "cpu".
The pretrained models that are uploaded can be used for evaluation of results. However, if desired the trauining can also be done, the only prerequisities would be ensuring the required data files are available in data folder and the preprocessing steps to generate the "data_arrays.npz" have successfully completed.

Subsequent functions for testing the model and capturing the results in the "results" folder are defined.

In [97]:
class birnn_concat_time_delta(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(birnn_concat_time_delta, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)

      # GRU layers
      self.gru_dp_fw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_fw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)
      self.gru_dp_bw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_bw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*(self.embed_dp_dim+1), 1)
      self.fc_cp  = nn.Linear(2*(self.embed_cp_dim+1), 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Compute time delta
      ## output dim: batch_size x seq_len
      dp_t_delta_fw = abs_time_to_delta(dp_t)
      cp_t_delta_fw = abs_time_to_delta(cp_t)
      dp_t_delta_bw = abs_time_to_delta(torch.flip(dp_t, [1]))
      cp_t_delta_bw = abs_time_to_delta(torch.flip(cp_t, [1]))    
    
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = self.embed_dp(dp)
      embedded_cp_fw = self.embed_cp(cp)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      
      # Concatate with time
      ## output dim: batch_size x seq_len x (embedding_dim+1)
      concat_dp_fw = torch.cat((embedded_dp_fw, torch.unsqueeze(dp_t_delta_fw, dim=-1)), dim=-1)
      concat_cp_fw = torch.cat((embedded_cp_fw, torch.unsqueeze(cp_t_delta_fw, dim=-1)), dim=-1)
      concat_dp_bw = torch.cat((embedded_dp_bw, torch.unsqueeze(dp_t_delta_bw, dim=-1)), dim=-1)
      concat_cp_bw = torch.cat((embedded_cp_bw, torch.unsqueeze(cp_t_delta_bw, dim=-1)), dim=-1)
      ## Dropout
      concat_dp_fw = self.dropout(concat_dp_fw)
      concat_cp_fw = self.dropout(concat_cp_fw)
      concat_dp_bw = self.dropout(concat_dp_bw)
      concat_cp_bw = self.dropout(concat_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x (embedding_dim+1)
      ## output dim rnn_hidden: batch_size x 1 x (embedding_dim+1)
      rnn_dp_fw, rnn_hidden_dp_fw = self.gru_dp_fw(concat_dp_fw)
      rnn_cp_fw, rnn_hidden_cp_fw = self.gru_cp_fw(concat_cp_fw)
      rnn_dp_bw, rnn_hidden_dp_bw = self.gru_dp_bw(concat_dp_bw)
      rnn_cp_bw, rnn_hidden_cp_bw = self.gru_cp_bw(concat_cp_bw)      
      ## output dim rnn_hidden: batch_size x (embedding_dim+1)
      rnn_hidden_dp_fw = rnn_hidden_dp_fw.view(-1, self.embed_dp_dim+1)
      rnn_hidden_cp_fw = rnn_hidden_cp_fw.view(-1, self.embed_cp_dim+1)
      rnn_hidden_dp_bw = rnn_hidden_dp_bw.view(-1, self.embed_dp_dim+1)
      rnn_hidden_cp_bw = rnn_hidden_cp_bw.view(-1, self.embed_cp_dim+1)
      ## concatenate forward and backward: batch_size x 2*(embedding_dim+1)
      rnn_hidden_dp = torch.cat((rnn_hidden_dp_fw, rnn_hidden_dp_bw), dim=-1)
      rnn_hidden_cp = torch.cat((rnn_hidden_cp_fw, rnn_hidden_cp_bw), dim=-1)
      
      # Scores
      score_dp = self.fc_dp(self.dropout(rnn_hidden_dp))
      score_cp = self.fc_cp(self.dropout(rnn_hidden_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []
    

In [98]:
class GRUOdeDecay(nn.Module):
  """
  GRU RNN module where the hidden state decays according to an ODE.
  (see Rubanova et al. 2019, Latent ODEs for Irregularly-Sampled Time Series)
  
  Args:
    inputs: A `Tensor` with embeddings in the last dimension.
    times: A `Tensor` with the same shape as inputs containing the recorded times (but no embedding dimension).

  Returns:
    outs: Hidden states of the RNN.
  """
  def __init__(self, input_size, hidden_size, bias=True):
    super(GRUOdeDecay, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.gru_cell = nn.GRUCell(input_size, hidden_size)
    self.decays = nn.Parameter(torch.Tensor(hidden_size)) # exponential decays vector
    
    # ODE
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    self.ode_net = ODENet(self.device, self.input_size, self.input_size, output_dim=self.input_size, augment_dim=0, time_dependent=False, non_linearity='softplus', tol=1e-3, adjoint=True)
  
  def forward(self, inputs, times):
    # initializing and then calling cuda() later isn't working for some reason
    if torch.cuda.is_available():
      hn = torch.zeros(inputs.size(0), self.hidden_size).cuda() # batch_size x hidden_size
      outs = torch.zeros(inputs.size(0), inputs.size(1), self.hidden_size).cuda() # batch_size x seq_len x hidden_size
    else:
      hn = torch.zeros(inputs.size(0), self.hidden_size) # batch_size x hidden_size
      outs = torch.zeros(inputs.size(0), inputs.size(1), self.hidden_size) # batch_size x seq_len x hidden_size

    # this is slow
    for seq in range(inputs.size(1)):
      hn = self.gru_cell(inputs[:,seq,:], hn)
      outs[:,seq,:] = hn
      
      times_unique, inverse_indices = torch.unique(times[:,seq], sorted=True, return_inverse=True)
      if times_unique.size(0) > 1:
        hn = self.ode_net(hn, times_unique)
        hn = hn[inverse_indices, torch.arange(0, inverse_indices.size(0)), :]
    return outs


In [99]:
class Attention(torch.nn.Module):
  """
  Dot-product attention module.
  
  Args:
    inputs: A `Tensor` with embeddings in the last dimension.
    mask: A `Tensor`. Dimensions are the same as inputs but without the embedding dimension.
      Values are 0 for 0-padding in the input and 1 elsewhere.

  Returns:
    outputs: The input `Tensor` whose embeddings in the last dimension have undergone a weighted average.
      The second-last dimension of the `Tensor` is removed.
    attention_weights: weights given to each embedding.
  """
  def __init__(self, embedding_dim):
    super(Attention, self).__init__()
    self.context = nn.Parameter(torch.Tensor(embedding_dim)) # context vector
    self.linear_hidden = nn.Linear(embedding_dim, embedding_dim)
    self.reset_parameters()
    
  def reset_parameters(self):
    nn.init.normal_(self.context)

  def forward(self, inputs, mask):
    # Hidden representation of embeddings (no change in dimensions)
    hidden = torch.tanh(self.linear_hidden(inputs))
    # Compute weight of each embedding
    importance = torch.sum(hidden * self.context, dim=-1)
    importance = importance.masked_fill(mask == 0, -1e9)
    # Softmax so that weights sum up to one
    attention_weights = F.softmax(importance, dim=-1)
    # Weighted sum of embeddings
    weighted_projection = inputs * torch.unsqueeze(attention_weights, dim=-1)
    # Output
    outputs = torch.sum(weighted_projection, dim=-2)
    return outputs, attention_weights


In [100]:
MAX_NUM_STEPS = 1000 

class ODEFunc(nn.Module):
    """MLP modeling the derivative of ODE system.
    Parameters
    ----------
    device : torch.device
    data_dim : int
        Dimension of data.
    hidden_dim : int
        Dimension of hidden layers.
    augment_dim: int
        Dimension of augmentation. If 0 does not augment ODE, otherwise augments
        it with augment_dim dimensions.
    time_dependent : bool
        If True adds time as input, making ODE time dependent.
    non_linearity : string
        One of 'relu' and 'softplus'
    """
    def __init__(self, device, data_dim, hidden_dim, augment_dim=0,
                 time_dependent=False, non_linearity='relu'):
        super(ODEFunc, self).__init__()
        self.device = device
        self.augment_dim = augment_dim
        self.data_dim = data_dim
        self.input_dim = data_dim + augment_dim
        self.hidden_dim = hidden_dim
        self.nfe = 0  # Number of function evaluations
        self.time_dependent = time_dependent

        if time_dependent:
            self.fc1 = nn.Linear(self.input_dim + 1, hidden_dim)
        else:
            self.fc1 = nn.Linear(self.input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, self.input_dim)

        if non_linearity == 'relu':
            self.non_linearity = nn.ReLU(inplace=True)
        elif non_linearity == 'softplus':
            self.non_linearity = nn.Softplus()

    def forward(self, t, x):
        """
        Parameters
        ----------
        t : torch.Tensor
            Current time. Shape (1,).
        x : torch.Tensor
            Shape (batch_size, input_dim)
        """
        # Forward pass of model corresponds to one function evaluation, so
        # increment counter
        self.nfe += 1
        if self.time_dependent:
            # Shape (batch_size, 1)
            t_vec = torch.ones(x.shape[0], 1).to(self.device) * t
            # Shape (batch_size, data_dim + 1)
            t_and_x = torch.cat([t_vec, x], 1)
            # Shape (batch_size, hidden_dim)
            out = self.fc1(t_and_x)
        else:
            out = self.fc1(x)
        out = self.non_linearity(out)
        out = self.fc2(out)
        out = self.non_linearity(out)
        out = self.fc3(out)
        return out


class ODEBlock(nn.Module):
    """Solves ODE defined by odefunc.
    Parameters
    ----------
    device : torch.device
    odefunc : ODEFunc instance or anode.conv_models.ConvODEFunc instance
        Function defining dynamics of system.
    is_conv : bool
        If True, treats odefunc as a convolutional model.
    tol : float
        Error tolerance.
    adjoint : bool
        If True calculates gradient with adjoint method, otherwise
        backpropagates directly through operations of ODE solver.
    """
    def __init__(self, device, odefunc, is_conv=False, tol=1e-3, adjoint=False):
        super(ODEBlock, self).__init__()
        self.adjoint = adjoint
        self.device = device
        self.is_conv = is_conv
        self.odefunc = odefunc
        self.tol = tol

    def forward(self, x, eval_times=None):
        """Solves ODE starting from x.
        Parameters
        ----------
        x : torch.Tensor
            Shape (batch_size, self.odefunc.data_dim)
        eval_times : None or torch.Tensor
            If None, returns solution of ODE at final time t=1. If torch.Tensor
            then returns full ODE trajectory evaluated at points in eval_times.
        """
        # Forward pass corresponds to solving ODE, so reset number of function
        # evaluations counter
        self.odefunc.nfe = 0
        
        if eval_times is None:
            integration_time = torch.tensor([0, 1]).float().type_as(x)
        else:
            integration_time = eval_times.type_as(x)


        if self.odefunc.augment_dim > 0:
            if self.is_conv:
                # Add augmentation
                batch_size, channels, height, width = x.shape
                aug = torch.zeros(batch_size, self.odefunc.augment_dim,
                                  height, width).to(self.device)
                # Shape (batch_size, channels + augment_dim, height, width)
                x_aug = torch.cat([x, aug], 1)
            else:
                # Add augmentation
                aug = torch.zeros(x.shape[0], self.odefunc.augment_dim).to(self.device)
                # Shape (batch_size, data_dim + augment_dim)
                x_aug = torch.cat([x, aug], 1)
        else:
            x_aug = x

        if self.adjoint:
            out = odeint_adjoint(self.odefunc, x_aug, integration_time,
                                 rtol=self.tol, atol=self.tol, method='euler',
                                 options={'max_num_steps': MAX_NUM_STEPS})
        else:
            out = odeint(self.odefunc, x_aug, integration_time,
                         rtol=self.tol, atol=self.tol, method='euler',
                         options={'max_num_steps': MAX_NUM_STEPS})

        if eval_times is None:
            return out[1]  # Return only final time
        else:
            return out


class ODENet(nn.Module):
    """An ODEBlock followed by a Linear layer.
    Parameters
    ----------
    device : torch.device
    data_dim : int
        Dimension of data.
    hidden_dim : int
        Dimension of hidden layers.
    output_dim : int
        Dimension of output after hidden layer. Should be 1 for regression or
        num_classes for classification.
    augment_dim: int
        Dimension of augmentation. If 0 does not augment ODE, otherwise augments
        it with augment_dim dimensions.
    time_dependent : bool
        If True adds time as input, making ODE time dependent.
    non_linearity : string
        One of 'relu' and 'softplus'
    tol : float
        Error tolerance.
    adjoint : bool
        If True calculates gradient with adjoint method, otherwise
        backpropagates directly through operations of ODE solver.
    """
    def __init__(self, device, data_dim, hidden_dim, output_dim=1,
                 augment_dim=0, time_dependent=False, non_linearity='relu',
                 tol=1e-3, adjoint=False):
        super(ODENet, self).__init__()
        self.device = device
        self.data_dim = data_dim
        self.hidden_dim = hidden_dim
        self.augment_dim = augment_dim
        self.output_dim = output_dim
        self.time_dependent = time_dependent
        self.tol = tol

        odefunc = ODEFunc(device, data_dim, hidden_dim, augment_dim,
                          time_dependent, non_linearity)

        self.odeblock = ODEBlock(device, odefunc, tol=tol, adjoint=adjoint)

    def forward(self, x, eval_times=None):
        features = self.odeblock(x, eval_times)
        return features
        
class GRUExponentialDecay(nn.Module):
    """
    GRU RNN module where the hidden state decays exponentially
    (see e.g. Che et al. 2018, Recurrent Neural Networks for Multivariate Time Series
    with Missing Values).
    
    Args:
    inputs: A `Tensor` with embeddings in the last dimension.
    times: A `Tensor` with the same shape as inputs containing the recorded times (but no embedding dimension).
    
    Returns:
    outs: Hidden states of the RNN.
    """
    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUExponentialDecay, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.gru_cell = nn.GRUCell(input_size, hidden_size)
        self.decays = nn.Parameter(torch.Tensor(hidden_size)) # exponential decays vector
    
    def forward(self, inputs, times):
    # initializing and then calling cuda() later isn't working for some reason
        if torch.cuda.is_available():
          hn = torch.zeros(inputs.size(0), self.hidden_size).cuda() # batch_size x hidden_size
          outs = torch.zeros(inputs.size(0), inputs.size(1), self.hidden_size).cuda() # batch_size x seq_len x hidden_size
        else:
          hn = torch.zeros(inputs.size(0), self.hidden_size) # batch_size x hidden_size
          outs = torch.zeros(inputs.size(0), inputs.size(1), self.hidden_size) # batch_size x seq_len x hidden_size
    
    # this is slow
        for seq in range(inputs.size(1)):
          hn = self.gru_cell(inputs[:,seq,:], hn)
          outs[:,seq,:] = hn
          hn = hn*torch.exp(-torch.clamp(torch.unsqueeze(times[:,seq], dim=-1)*self.decays, min=0))
        return outs

In [101]:
class birnn_concat_time_delta_attention(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(birnn_concat_time_delta_attention, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)

      # GRU layers
      self.gru_dp_fw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_fw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)
      self.gru_dp_bw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_bw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)

      # Attention layers
      self.attention_dp = Attention(embedding_dim=2*(self.embed_dp_dim+1)) #+1 for the concatenated time
      self.attention_cp = Attention(embedding_dim=2*(self.embed_cp_dim+1))
            
      # Fully connected output
      self.fc_dp  = nn.Linear(2*(self.embed_dp_dim+1), 1)
      self.fc_cp  = nn.Linear(2*(self.embed_cp_dim+1), 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Compute time delta
      ## output dim: batch_size x seq_len
      dp_t_delta_fw = abs_time_to_delta(dp_t)
      cp_t_delta_fw = abs_time_to_delta(cp_t)
      dp_t_delta_bw = abs_time_to_delta(torch.flip(dp_t, [1]))
      cp_t_delta_bw = abs_time_to_delta(torch.flip(cp_t, [1]))    
    
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = self.embed_dp(dp)
      embedded_cp_fw = self.embed_cp(cp)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      
      # Concatate with time
      ## output dim: batch_size x seq_len x (embedding_dim+1)
      concat_dp_fw = torch.cat((embedded_dp_fw, torch.unsqueeze(dp_t_delta_fw, dim=-1)), dim=-1)
      concat_cp_fw = torch.cat((embedded_cp_fw, torch.unsqueeze(cp_t_delta_fw, dim=-1)), dim=-1)
      concat_dp_bw = torch.cat((embedded_dp_bw, torch.unsqueeze(dp_t_delta_bw, dim=-1)), dim=-1)
      concat_cp_bw = torch.cat((embedded_cp_bw, torch.unsqueeze(cp_t_delta_bw, dim=-1)), dim=-1)
      ## Dropout
      concat_dp_fw = self.dropout(concat_dp_fw)
      concat_cp_fw = self.dropout(concat_cp_fw)
      concat_dp_bw = self.dropout(concat_dp_bw)
      concat_cp_bw = self.dropout(concat_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x (embedding_dim+1)
      ## output dim rnn_hidden: batch_size x 1 x (embedding_dim+1)
      rnn_dp_fw, rnn_hidden_dp_fw = self.gru_dp_fw(concat_dp_fw)
      rnn_cp_fw, rnn_hidden_cp_fw = self.gru_cp_fw(concat_cp_fw)
      rnn_dp_bw, rnn_hidden_dp_bw = self.gru_dp_bw(concat_dp_bw)
      rnn_cp_bw, rnn_hidden_cp_bw = self.gru_cp_bw(concat_cp_bw)      
      # concatenate forward and backward
      ## output dim: batch_size x seq_len x 2*(embedding_dim+1)
      rnn_dp = torch.cat((rnn_dp_fw, torch.flip(rnn_dp_bw, [1])), dim=-1)
      rnn_cp = torch.cat((rnn_cp_fw, torch.flip(rnn_cp_bw, [1])), dim=-1)

      # Attention
      ## output dim: batch_size x 2*(embedding_dim+1)
      attended_dp, weights_dp = self.attention_dp(rnn_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(rnn_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []


In [102]:
class birnn_ode_decay(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(birnn_ode_decay, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))+1
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))+1

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)

      # GRU layers
      self.gru_dp_fw = GRUOdeDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_fw = GRUOdeDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)
      self.gru_dp_bw = GRUOdeDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_bw = GRUOdeDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*self.embed_dp_dim, 1)
      self.fc_cp  = nn.Linear(2*self.embed_cp_dim, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Compute time delta
      ## output dim: batch_size x seq_len
      dp_t_delta_fw = abs_time_to_delta(dp_t)
      cp_t_delta_fw = abs_time_to_delta(cp_t)
      ## Round
      dp_t_delta_fw = torch.round(100*dp_t_delta_fw)/100
      cp_t_delta_fw = torch.round(100*cp_t_delta_fw)/100            
      dp_t_delta_bw = abs_time_to_delta(torch.flip(dp_t, [1]))
      cp_t_delta_bw = abs_time_to_delta(torch.flip(cp_t, [1]))    
    
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = self.embed_dp(dp)
      embedded_cp_fw = self.embed_cp(cp)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      ## Dropout
      embedded_dp_fw = self.dropout(embedded_dp_fw)
      embedded_cp_fw = self.dropout(embedded_cp_fw)
      embedded_dp_bw = self.dropout(embedded_dp_bw)
      embedded_cp_bw = self.dropout(embedded_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x embedding_dim
      rnn_dp_fw = self.gru_dp_fw(embedded_dp_fw, dp_t_delta_fw)
      rnn_cp_fw = self.gru_cp_fw(embedded_cp_fw, cp_t_delta_fw)
      rnn_dp_bw = self.gru_dp_bw(embedded_dp_bw, dp_t_delta_bw)
      rnn_cp_bw = self.gru_cp_bw(embedded_cp_bw, cp_t_delta_bw)      
      ## output dim rnn_hidden: batch_size x embedding_dim
      rnn_dp_fw = rnn_dp_fw[:,-1,:]
      rnn_cp_fw = rnn_cp_fw[:,-1,:]
      rnn_dp_bw = rnn_dp_bw[:,-1,:]
      rnn_cp_bw = rnn_cp_bw[:,-1,:]
      ## concatenate forward and backward: batch_size x 2*embedding_dim
      rnn_dp = torch.cat((rnn_dp_fw, rnn_dp_bw), dim=-1)
      rnn_cp = torch.cat((rnn_cp_fw, rnn_cp_bw), dim=-1)
      
      # Scores
      score_dp = self.fc_dp(self.dropout(rnn_dp))
      score_cp = self.fc_cp(self.dropout(rnn_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []      
      

In [103]:
class birnn_time_decay_attention(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(birnn_time_decay_attention, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))+1
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))+1

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)

      # GRU layers
      self.gru_dp_fw = GRUExponentialDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_fw = GRUExponentialDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)
      self.gru_dp_bw = GRUExponentialDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_bw = GRUExponentialDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)

      # Attention layers
      self.attention_dp = Attention(embedding_dim=2*self.embed_dp_dim)
      self.attention_cp = Attention(embedding_dim=2*self.embed_cp_dim)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*self.embed_dp_dim, 1)
      self.fc_cp  = nn.Linear(2*self.embed_cp_dim, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Compute time delta
      ## output dim: batch_size x seq_len
      dp_t_delta_fw = abs_time_to_delta(dp_t)
      cp_t_delta_fw = abs_time_to_delta(cp_t)
      dp_t_delta_bw = abs_time_to_delta(torch.flip(dp_t, [1]))
      cp_t_delta_bw = abs_time_to_delta(torch.flip(cp_t, [1]))    
    
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = self.embed_dp(dp)
      embedded_cp_fw = self.embed_cp(cp)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      ## Dropout
      embedded_dp_fw = self.dropout(embedded_dp_fw)
      embedded_cp_fw = self.dropout(embedded_cp_fw)
      embedded_dp_bw = self.dropout(embedded_dp_bw)
      embedded_cp_bw = self.dropout(embedded_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x embedding_dim
      rnn_dp_fw = self.gru_dp_fw(embedded_dp_fw, dp_t_delta_fw)
      rnn_cp_fw = self.gru_cp_fw(embedded_cp_fw, cp_t_delta_fw)
      rnn_dp_bw = self.gru_dp_bw(embedded_dp_bw, dp_t_delta_bw)
      rnn_cp_bw = self.gru_cp_bw(embedded_cp_bw, cp_t_delta_bw)
      # concatenate forward and backward
      ## output dim: batch_size x seq_len x 2*embedding_dim
      rnn_dp = torch.cat((rnn_dp_fw, torch.flip(rnn_dp_bw, [1])), dim=-1)
      rnn_cp = torch.cat((rnn_cp_fw, torch.flip(rnn_cp_bw, [1])), dim=-1)

      # Attention
      ## output dim: batch_size x 2*embedding_dim
      attended_dp, weights_dp = self.attention_dp(rnn_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(rnn_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []
      

In [104]:
class birnn_time_decay(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(birnn_time_decay, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))+1
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))+1

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)

      # GRU layers
      self.gru_dp_fw = GRUExponentialDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_fw = GRUExponentialDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)
      self.gru_dp_bw = GRUExponentialDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_bw = GRUExponentialDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*self.embed_dp_dim, 1)
      self.fc_cp  = nn.Linear(2*self.embed_cp_dim, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Compute time delta
      ## output dim: batch_size x seq_len
      dp_t_delta_fw = abs_time_to_delta(dp_t)
      cp_t_delta_fw = abs_time_to_delta(cp_t)
      dp_t_delta_bw = abs_time_to_delta(torch.flip(dp_t, [1]))
      cp_t_delta_bw = abs_time_to_delta(torch.flip(cp_t, [1]))    
    
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = self.embed_dp(dp)
      embedded_cp_fw = self.embed_cp(cp)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      ## Dropout
      embedded_dp_fw = self.dropout(embedded_dp_fw)
      embedded_cp_fw = self.dropout(embedded_cp_fw)
      embedded_dp_bw = self.dropout(embedded_dp_bw)
      embedded_cp_bw = self.dropout(embedded_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x embedding_dim
      rnn_dp_fw = self.gru_dp_fw(embedded_dp_fw, dp_t_delta_fw)
      rnn_cp_fw = self.gru_cp_fw(embedded_cp_fw, cp_t_delta_fw)
      rnn_dp_bw = self.gru_dp_bw(embedded_dp_bw, dp_t_delta_bw)
      rnn_cp_bw = self.gru_cp_bw(embedded_cp_bw, cp_t_delta_bw)      
      ## output dim rnn_hidden: batch_size x embedding_dim
      rnn_dp_fw = rnn_dp_fw[:,-1,:]
      rnn_cp_fw = rnn_cp_fw[:,-1,:]
      rnn_dp_bw = rnn_dp_bw[:,-1,:]
      rnn_cp_bw = rnn_cp_bw[:,-1,:]
      ## concatenate forward and backward: batch_size x 2*embedding_dim
      rnn_dp = torch.cat((rnn_dp_fw, rnn_dp_bw), dim=-1)
      rnn_cp = torch.cat((rnn_cp_fw, rnn_cp_bw), dim=-1)
      
      # Scores
      score_dp = self.fc_dp(self.dropout(rnn_dp))
      score_cp = self.fc_cp(self.dropout(rnn_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []
      

In [105]:
 class ode_attention(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(ode_attention, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(2*np.ceil(num_dp_codes**0.25))+1
      self.embed_cp_dim = int(2*np.ceil(num_cp_codes**0.25))+1

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)
      
      # ODE layers
      self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
      self.ode_dp = ODENet(self.device, self.embed_dp_dim, self.embed_dp_dim, output_dim=self.embed_dp_dim, augment_dim=0, time_dependent=False, non_linearity='softplus', tol=1e-3, adjoint=True)
      self.ode_cp = ODENet(self.device, self.embed_cp_dim, self.embed_cp_dim, output_dim=self.embed_cp_dim, augment_dim=0, time_dependent=False, non_linearity='softplus', tol=1e-3, adjoint=True)
      
      # Attention layers
      self.attention_dp = Attention(embedding_dim=self.embed_dp_dim)
      self.attention_cp = Attention(embedding_dim=self.embed_cp_dim)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(self.embed_dp_dim, 1)
      self.fc_cp  = nn.Linear(self.embed_cp_dim, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp = self.embed_dp(dp)
      embedded_cp = self.embed_cp(cp)
      
      # ODE
      ## Round times
      dp_t = torch.round(100*dp_t)/100
      cp_t = torch.round(100*cp_t)/100
      
      embedded_dp_long = embedded_dp.view(-1, self.embed_dp_dim)
      dp_t_long = dp_t.view(-1)
      dp_t_long_unique, inverse_indices = torch.unique(dp_t_long, sorted=True, return_inverse=True)
      ode_dp_long = self.ode_dp(embedded_dp_long, dp_t_long_unique)
      ode_dp_long = ode_dp_long[inverse_indices, torch.arange(0, inverse_indices.size(0)), :]
      ode_dp = ode_dp_long.view(dp.size(0), dp.size(1), self.embed_dp_dim)

      embedded_cp_long = embedded_cp.view(-1, self.embed_cp_dim)
      cp_t_long = cp_t.view(-1)
      cp_t_long_unique, inverse_indices = torch.unique(cp_t_long, sorted=True, return_inverse=True)
      ode_cp_long = self.ode_cp(embedded_cp_long, cp_t_long_unique)
      ode_cp_long = ode_cp_long[inverse_indices, torch.arange(0, inverse_indices.size(0)), :]
      ode_cp = ode_cp_long.view(cp.size(0), cp.size(1), self.embed_cp_dim)

      ## Dropout
      ode_dp = self.dropout(ode_dp)
      ode_cp = self.dropout(ode_cp)
      
      # Attention
      ## output dim: batch_size x (embedding_dim+1)
      attended_dp, weights_dp = self.attention_dp(ode_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(ode_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []


In [106]:
 class ode_birnn(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(ode_birnn, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))+1
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))+1

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)
      
      # ODE layers
      self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
      self.ode_dp = ODENet(self.device, self.embed_dp_dim, self.embed_dp_dim, output_dim=self.embed_dp_dim, augment_dim=0, time_dependent=False, non_linearity='softplus', tol=1e-3, adjoint=True)
      self.ode_cp = ODENet(self.device, self.embed_cp_dim, self.embed_cp_dim, output_dim=self.embed_cp_dim, augment_dim=0, time_dependent=False, non_linearity='softplus', tol=1e-3, adjoint=True)

      # GRU layers
      self.gru_dp_fw = nn.GRU(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim, num_layers=1, batch_first=True)
      self.gru_cp_fw = nn.GRU(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim, num_layers=1, batch_first=True)
      self.gru_dp_bw = nn.GRU(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim, num_layers=1, batch_first=True)
      self.gru_cp_bw = nn.GRU(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim, num_layers=1, batch_first=True)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*self.embed_dp_dim, 1)
      self.fc_cp  = nn.Linear(2*self.embed_cp_dim, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp = self.embed_dp(dp)
      embedded_cp = self.embed_cp(cp)
      
      # ODE
      ## Round times
      dp_t = torch.round(100*dp_t)/100
      cp_t = torch.round(100*cp_t)/100
      
      embedded_dp_long = embedded_dp.view(-1, self.embed_dp_dim)
      dp_t_long = dp_t.view(-1)
      dp_t_long_unique, inverse_indices = torch.unique(dp_t_long, sorted=True, return_inverse=True)
      ode_dp_long = self.ode_dp(embedded_dp_long, dp_t_long_unique)
      ode_dp_long = ode_dp_long[inverse_indices, torch.arange(0, inverse_indices.size(0)), :]
      ode_dp = ode_dp_long.view(dp.size(0), dp.size(1), self.embed_dp_dim)

      embedded_cp_long = embedded_cp.view(-1, self.embed_cp_dim)
      cp_t_long = cp_t.view(-1)
      cp_t_long_unique, inverse_indices = torch.unique(cp_t_long, sorted=True, return_inverse=True)
      ode_cp_long = self.ode_cp(embedded_cp_long, cp_t_long_unique)
      ode_cp_long = ode_cp_long[inverse_indices, torch.arange(0, inverse_indices.size(0)), :]
      ode_cp = ode_cp_long.view(cp.size(0), cp.size(1), self.embed_cp_dim)
      
      ## Dropout
      ode_dp = self.dropout(ode_dp)
      ode_cp = self.dropout(ode_cp)

      # Forward and backward sequences
      ## output dim: batch_size x seq_len x embedding_dim
      ode_dp_fw = ode_dp
      ode_cp_fw = ode_cp
      ode_dp_bw = torch.flip(ode_dp_fw, [1])
      ode_cp_bw = torch.flip(ode_cp_fw, [1])
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x embedding_dim
      ## output dim rnn_hidden: batch_size x 1 x embedding_dim
      rnn_dp_fw, rnn_hidden_dp_fw = self.gru_dp_fw(ode_dp_fw)
      rnn_cp_fw, rnn_hidden_cp_fw = self.gru_cp_fw(ode_cp_fw)
      rnn_dp_bw, rnn_hidden_dp_bw = self.gru_dp_bw(ode_dp_bw)
      rnn_cp_bw, rnn_hidden_cp_bw = self.gru_cp_bw(ode_cp_bw)      
      ## output dim rnn_hidden: batch_size x embedding_dim
      rnn_hidden_dp_fw = rnn_hidden_dp_fw.view(-1, self.embed_dp_dim)
      rnn_hidden_cp_fw = rnn_hidden_cp_fw.view(-1, self.embed_cp_dim)
      rnn_hidden_dp_bw = rnn_hidden_dp_bw.view(-1, self.embed_dp_dim)
      rnn_hidden_cp_bw = rnn_hidden_cp_bw.view(-1, self.embed_cp_dim)
      ## concatenate forward and backward: batch_size x 2*embedding_dim
      rnn_hidden_dp = torch.cat((rnn_hidden_dp_fw, rnn_hidden_dp_bw), dim=-1)
      rnn_hidden_cp = torch.cat((rnn_hidden_cp_fw, rnn_hidden_cp_bw), dim=-1)
      
      # Scores
      score_dp = self.fc_dp(self.dropout(rnn_hidden_dp))
      score_cp = self.fc_cp(self.dropout(rnn_hidden_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []


In [107]:
 class ode_birnn_attention(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(ode_birnn_attention, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))+1
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))+1

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)
      
      # ODE layers
      self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
      self.ode_dp = ODENet(self.device, self.embed_dp_dim, self.embed_dp_dim, output_dim=self.embed_dp_dim, augment_dim=0, time_dependent=False, non_linearity='softplus', tol=1e-3, adjoint=True)
      self.ode_cp = ODENet(self.device, self.embed_cp_dim, self.embed_cp_dim, output_dim=self.embed_cp_dim, augment_dim=0, time_dependent=False, non_linearity='softplus', tol=1e-3, adjoint=True)

      # GRU layers
      self.gru_dp_fw = nn.GRU(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim, num_layers=1, batch_first=True)
      self.gru_cp_fw = nn.GRU(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim, num_layers=1, batch_first=True)
      self.gru_dp_bw = nn.GRU(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim, num_layers=1, batch_first=True)
      self.gru_cp_bw = nn.GRU(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim, num_layers=1, batch_first=True)

      # Attention layers
      self.attention_dp = Attention(embedding_dim=2*self.embed_dp_dim)
      self.attention_cp = Attention(embedding_dim=2*self.embed_cp_dim)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*self.embed_dp_dim, 1)
      self.fc_cp  = nn.Linear(2*self.embed_cp_dim, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp = self.embed_dp(dp)
      embedded_cp = self.embed_cp(cp)
      
      # ODE
      ## Round times
      dp_t = torch.round(100*dp_t)/100
      cp_t = torch.round(100*cp_t)/100
      
      embedded_dp_long = embedded_dp.view(-1, self.embed_dp_dim)
      dp_t_long = dp_t.view(-1)
      dp_t_long_unique, inverse_indices = torch.unique(dp_t_long, sorted=True, return_inverse=True)
      ode_dp_long = self.ode_dp(embedded_dp_long, dp_t_long_unique)
      ode_dp_long = ode_dp_long[inverse_indices, torch.arange(0, inverse_indices.size(0)), :]
      ode_dp = ode_dp_long.view(dp.size(0), dp.size(1), self.embed_dp_dim)

      embedded_cp_long = embedded_cp.view(-1, self.embed_cp_dim)
      cp_t_long = cp_t.view(-1)
      cp_t_long_unique, inverse_indices = torch.unique(cp_t_long, sorted=True, return_inverse=True)
      ode_cp_long = self.ode_cp(embedded_cp_long, cp_t_long_unique)
      ode_cp_long = ode_cp_long[inverse_indices, torch.arange(0, inverse_indices.size(0)), :]
      ode_cp = ode_cp_long.view(cp.size(0), cp.size(1), self.embed_cp_dim)
      
      ## Dropout
      ode_dp = self.dropout(ode_dp)
      ode_cp = self.dropout(ode_cp)

      # Forward and backward sequences
      ## output dim: batch_size x seq_len x embedding_dim
      ode_dp_fw = ode_dp
      ode_cp_fw = ode_cp
      ode_dp_bw = torch.flip(ode_dp_fw, [1])
      ode_cp_bw = torch.flip(ode_cp_fw, [1])
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x embedding_dim
      ## output dim rnn_hidden: batch_size x 1 x embedding_dim
      rnn_dp_fw, rnn_hidden_dp_fw = self.gru_dp_fw(ode_dp_fw)
      rnn_cp_fw, rnn_hidden_cp_fw = self.gru_cp_fw(ode_cp_fw)
      rnn_dp_bw, rnn_hidden_dp_bw = self.gru_dp_bw(ode_dp_bw)
      rnn_cp_bw, rnn_hidden_cp_bw = self.gru_cp_bw(ode_cp_bw)      
      # concatenate forward and backward
      ## output dim: batch_size x seq_len x 2*embedding_dim
      rnn_dp = torch.cat((rnn_dp_fw, torch.flip(rnn_dp_bw, [1])), dim=-1)
      rnn_cp = torch.cat((rnn_cp_fw, torch.flip(rnn_cp_bw, [1])), dim=-1)

      # Attention
      ## output dim: batch_size x 2*embedding_dim
      attended_dp, weights_dp = self.attention_dp(rnn_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(rnn_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []


In [108]:
class birnn_ode_decay_attention(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(birnn_ode_decay_attention, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))+1
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))+1

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)

      # GRU layers
      self.gru_dp_fw = GRUOdeDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_fw = GRUOdeDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)
      self.gru_dp_bw = GRUOdeDecay(input_size=self.embed_dp_dim, hidden_size=self.embed_dp_dim)
      self.gru_cp_bw = GRUOdeDecay(input_size=self.embed_cp_dim, hidden_size=self.embed_cp_dim)

      # Attention layers
      self.attention_dp = Attention(embedding_dim=2*self.embed_dp_dim)
      self.attention_cp = Attention(embedding_dim=2*self.embed_cp_dim)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*self.embed_dp_dim, 1)
      self.fc_cp  = nn.Linear(2*self.embed_cp_dim, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Compute time delta
      ## output dim: batch_size x seq_len
      dp_t_delta_fw = abs_time_to_delta(dp_t)
      cp_t_delta_fw = abs_time_to_delta(cp_t)
      ## Round
      dp_t_delta_fw = torch.round(100*dp_t_delta_fw)/100
      cp_t_delta_fw = torch.round(100*cp_t_delta_fw)/100      
      dp_t_delta_bw = abs_time_to_delta(torch.flip(dp_t, [1]))
      cp_t_delta_bw = abs_time_to_delta(torch.flip(cp_t, [1]))    
    
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = self.embed_dp(dp)
      embedded_cp_fw = self.embed_cp(cp)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      ## Dropout
      embedded_dp_fw = self.dropout(embedded_dp_fw)
      embedded_cp_fw = self.dropout(embedded_cp_fw)
      embedded_dp_bw = self.dropout(embedded_dp_bw)
      embedded_cp_bw = self.dropout(embedded_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x embedding_dim
      rnn_dp_fw = self.gru_dp_fw(embedded_dp_fw, dp_t_delta_fw)
      rnn_cp_fw = self.gru_cp_fw(embedded_cp_fw, cp_t_delta_fw)
      rnn_dp_bw = self.gru_dp_bw(embedded_dp_bw, dp_t_delta_bw)
      rnn_cp_bw = self.gru_cp_bw(embedded_cp_bw, cp_t_delta_bw)
      # concatenate forward and backward
      ## output dim: batch_size x seq_len x 2*embedding_dim
      rnn_dp = torch.cat((rnn_dp_fw, torch.flip(rnn_dp_bw, [1])), dim=-1)
      rnn_cp = torch.cat((rnn_cp_fw, torch.flip(rnn_cp_bw, [1])), dim=-1)

      # Attention
      ## output dim: batch_size x 2*embedding_dim
      attended_dp, weights_dp = self.attention_dp(rnn_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(rnn_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []

In [109]:
class attention_concat_time(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(attention_concat_time, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(2*np.ceil(num_dp_codes**0.25))
      self.embed_cp_dim = int(2*np.ceil(num_cp_codes**0.25))

      # Embedding layers
      self.embed_dp = nn.Embedding(num_embeddings=num_dp_codes, embedding_dim=self.embed_dp_dim, padding_idx=0)
      self.embed_cp = nn.Embedding(num_embeddings=num_cp_codes, embedding_dim=self.embed_cp_dim, padding_idx=0)
      
      # Attention layers
      self.attention_dp = Attention(embedding_dim=self.embed_dp_dim+1) #+1 for the concatenated time
      self.attention_cp = Attention(embedding_dim=self.embed_cp_dim+1)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(self.embed_dp_dim+1, 1)
      self.fc_cp  = nn.Linear(self.embed_cp_dim+1, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp = self.embed_dp(dp)
      embedded_cp = self.embed_cp(cp)
      
      # Concatate with time
      ## output dim: batch_size x seq_len x (embedding_dim+1)
      concat_dp = torch.cat((embedded_dp, torch.unsqueeze(dp_t, dim=-1)), dim=-1)
      concat_cp = torch.cat((embedded_cp, torch.unsqueeze(cp_t, dim=-1)), dim=-1)
      ## Dropout
      concat_dp = self.dropout(concat_dp)
      concat_cp = self.dropout(concat_cp)
      
      # Attention
      ## output dim: batch_size x (embedding_dim+1)
      attended_dp, weights_dp = self.attention_dp(concat_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(concat_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []

In [110]:
 class mce_attention(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(mce_attention, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(2*np.ceil(num_dp_codes**0.25))
      self.embed_cp_dim = int(2*np.ceil(num_cp_codes**0.25))

      # Precomputed embedding weights
      data_dir = current_dir + '/data/'
      self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
      self.emb_weight_dp = torch.Tensor(np.load(data_dir + 'emb_weight_dp_13.npy')).to(self.device)
      self.emb_weight_cp = torch.Tensor(np.load(data_dir + 'emb_weight_cp_11.npy')).to(self.device)
      
      # Attention layers
      self.attention_dp = Attention(embedding_dim=self.embed_dp_dim+1) #+1 for the concatenated time
      self.attention_cp = Attention(embedding_dim=self.embed_cp_dim+1)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(self.embed_dp_dim+1, 1)
      self.fc_cp  = nn.Linear(self.embed_cp_dim+1, 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp = F.embedding(dp, self.emb_weight_dp, padding_idx=0)
      embedded_cp = F.embedding(cp, self.emb_weight_cp, padding_idx=0)
      ## Dropout
      embedded_dp = self.dropout(embedded_dp)
      embedded_cp = self.dropout(embedded_cp)
      
      # Attention
      ## output dim: batch_size x (embedding_dim+1)
      attended_dp, weights_dp = self.attention_dp(embedded_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(embedded_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []


In [111]:
 class mce_birnn(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(mce_birnn, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))

      # Precomputed embedding weights
      self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
      self.emb_weight_dp = torch.Tensor(np.load(data_dir + 'emb_weight_dp_7.npy')).to(self.device)
      self.emb_weight_cp = torch.Tensor(np.load(data_dir + 'emb_weight_cp_6.npy')).to(self.device)

      # GRU layers
      self.gru_dp_fw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_fw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)
      self.gru_dp_bw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_bw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)
      
      # Fully connected output
      self.fc_dp  = nn.Linear(2*(self.embed_dp_dim+1), 1)
      self.fc_cp  = nn.Linear(2*(self.embed_cp_dim+1), 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = F.embedding(dp, self.emb_weight_dp, padding_idx=0)
      embedded_cp_fw = F.embedding(cp, self.emb_weight_cp, padding_idx=0)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      ## Dropout
      embedded_dp_fw = self.dropout(embedded_dp_fw)
      embedded_cp_fw = self.dropout(embedded_cp_fw)
      embedded_dp_bw = self.dropout(embedded_dp_bw)
      embedded_cp_bw = self.dropout(embedded_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x (embedding_dim+1)
      ## output dim rnn_hidden: batch_size x 1 x (embedding_dim+1)
      rnn_dp_fw, rnn_hidden_dp_fw = self.gru_dp_fw(embedded_dp_fw)
      rnn_cp_fw, rnn_hidden_cp_fw = self.gru_cp_fw(embedded_cp_fw)
      rnn_dp_bw, rnn_hidden_dp_bw = self.gru_dp_bw(embedded_dp_bw)
      rnn_cp_bw, rnn_hidden_cp_bw = self.gru_cp_bw(embedded_cp_bw)      
      ## output dim rnn_hidden: batch_size x (embedding_dim+1)
      rnn_hidden_dp_fw = rnn_hidden_dp_fw.view(-1, self.embed_dp_dim+1)
      rnn_hidden_cp_fw = rnn_hidden_cp_fw.view(-1, self.embed_cp_dim+1)
      rnn_hidden_dp_bw = rnn_hidden_dp_bw.view(-1, self.embed_dp_dim+1)
      rnn_hidden_cp_bw = rnn_hidden_cp_bw.view(-1, self.embed_cp_dim+1)
      ## concatenate forward and backward: batch_size x 2*(embedding_dim+1)
      rnn_hidden_dp = torch.cat((rnn_hidden_dp_fw, rnn_hidden_dp_bw), dim=-1)
      rnn_hidden_cp = torch.cat((rnn_hidden_cp_fw, rnn_hidden_cp_bw), dim=-1)
      
      # Scores
      score_dp = self.fc_dp(self.dropout(rnn_hidden_dp))
      score_cp = self.fc_cp(self.dropout(rnn_hidden_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []


In [112]:
 class mce_birnn_attention(nn.Module):
    def __init__(self, num_static, num_dp_codes, num_cp_codes):
      super(mce_birnn_attention, self).__init__()
      
      # Embedding dimensions
      self.embed_dp_dim = int(np.ceil(num_dp_codes**0.25))
      self.embed_cp_dim = int(np.ceil(num_cp_codes**0.25))

      # Precomputed embedding weights
      self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
      self.emb_weight_dp = torch.Tensor(np.load(data_dir + 'emb_weight_dp_7.npy')).to(self.device)
      self.emb_weight_cp = torch.Tensor(np.load(data_dir + 'emb_weight_cp_6.npy')).to(self.device)

      # GRU layers
      self.gru_dp_fw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_fw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)
      self.gru_dp_bw = nn.GRU(input_size=self.embed_dp_dim+1, hidden_size=self.embed_dp_dim+1, num_layers=1, batch_first=True)
      self.gru_cp_bw = nn.GRU(input_size=self.embed_cp_dim+1, hidden_size=self.embed_cp_dim+1, num_layers=1, batch_first=True)

      # Attention layers
      self.attention_dp = Attention(embedding_dim=2*(self.embed_dp_dim+1)) #+1 for the concatenated time
      self.attention_cp = Attention(embedding_dim=2*(self.embed_cp_dim+1))
            
      # Fully connected output
      self.fc_dp  = nn.Linear(2*(self.embed_dp_dim+1), 1)
      self.fc_cp  = nn.Linear(2*(self.embed_cp_dim+1), 1)
      self.fc_all = nn.Linear(num_static + 2, 1)
      
      # Others
      self.dropout = nn.Dropout(p=0.5)

    def forward(self, stat, dp, cp, dp_t, cp_t):
      # Embedding
      ## output dim: batch_size x seq_len x embedding_dim
      embedded_dp_fw = F.embedding(dp, self.emb_weight_dp, padding_idx=0)
      embedded_cp_fw = F.embedding(cp, self.emb_weight_cp, padding_idx=0)
      embedded_dp_bw = torch.flip(embedded_dp_fw, [1])
      embedded_cp_bw = torch.flip(embedded_cp_fw, [1])
      ## Dropout
      embedded_dp_fw = self.dropout(embedded_dp_fw)
      embedded_cp_fw = self.dropout(embedded_cp_fw)
      embedded_dp_bw = self.dropout(embedded_dp_bw)
      embedded_cp_bw = self.dropout(embedded_cp_bw)
      
      # GRU
      ## output dim rnn:        batch_size x seq_len x (embedding_dim+1)
      ## output dim rnn_hidden: batch_size x 1 x (embedding_dim+1)
      rnn_dp_fw, rnn_hidden_dp_fw = self.gru_dp_fw(embedded_dp_fw)
      rnn_cp_fw, rnn_hidden_cp_fw = self.gru_cp_fw(embedded_cp_fw)
      rnn_dp_bw, rnn_hidden_dp_bw = self.gru_dp_bw(embedded_dp_bw)
      rnn_cp_bw, rnn_hidden_cp_bw = self.gru_cp_bw(embedded_cp_bw)   
      # concatenate forward and backward
      ## output dim: batch_size x seq_len x 2*(embedding_dim+1)
      rnn_dp = torch.cat((rnn_dp_fw, torch.flip(rnn_dp_bw, [1])), dim=-1)
      rnn_cp = torch.cat((rnn_cp_fw, torch.flip(rnn_cp_bw, [1])), dim=-1)

      # Attention
      ## output dim: batch_size x 2*(embedding_dim+1)
      attended_dp, weights_dp = self.attention_dp(rnn_dp, (dp > 0).float())
      attended_cp, weights_cp = self.attention_cp(rnn_cp, (cp > 0).float())
      
      # Scores
      score_dp = self.fc_dp(self.dropout(attended_dp))
      score_cp = self.fc_cp(self.dropout(attended_cp))

      # Concatenate to variable collection
      all = torch.cat((stat, score_dp, score_cp), dim=1)
      
      # Final linear projection
      out = self.fc_all(self.dropout(all)).squeeze()

      return out, []


In [113]:
from sklearn import linear_model

def logistic_regression(bootstrap_samples, np_seed):
    current_dir = os.getcwd()
    data_dir = current_dir + '/data/'
    print(data_dir)
    print('Loading data...')
    icu_pat = pd.read_pickle(data_dir + 'icu_pat_admit.pkl')
    
    print('Loading last vital signs measurements...')
    
    charts = pd.read_pickle(data_dir + 'charts_outputs_last_only.pkl')
    charts = charts.drop(columns=['CHARTTIME'])
    charts = pd.get_dummies(charts, columns = ['VALUECAT']).groupby('ICUSTAY_ID').sum()
    charts.drop(columns=['VALUECAT_CHART_BP_n', 'VALUECAT_CHART_BT_n', 'VALUECAT_CHART_GC_n', 'VALUECAT_CHART_HR_n', 'VALUECAT_CHART_RR_n', 'VALUECAT_CHART_UO_n'], inplace=True) # drop reference columns
    
    print('-----------------------------------------')
    
    print('Create array of static variables...')
    
    num_icu_stays = len(icu_pat['ICUSTAY_ID'])
    
    # static variables
    print('Create static array...')
    icu_pat = pd.get_dummies(icu_pat, columns = ['ADMISSION_LOCATION', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY'])
    icu_pat.drop(columns=['ADMISSION_LOCATION_Emergency Room Admit', 'INSURANCE_Medicare', 'MARITAL_STATUS_Married/Life Partner', 'ETHNICITY_White'], inplace=True) # drop reference columns
    
    # merge with last vital signs measurements
    icu_pat = pd.merge(icu_pat, charts, how='left', on='ICUSTAY_ID').fillna(0)
    
    static_columns = icu_pat.columns.str.contains('AGE|GENDER_M|LOS|NUM_RECENT_ADMISSIONS|ADMISSION_LOCATION|INSURANCE|MARITAL_STATUS|ETHNICITY|PRE_ICU_LOS|ELECTIVE_SURGERY|VALUECAT')
    static = icu_pat.loc[:, static_columns].values
    static_vars = icu_pat.loc[:, static_columns].columns.values.tolist()
    
    # classification label
    print('Create label array...')
    label = icu_pat.loc[:, 'POSITIVE'].values
    
    print('-----------------------------------------')
    
    print('Split data into train/validate/test...')
    # Split patients to avoid data leaks
    patients = icu_pat['SUBJECT_ID'].drop_duplicates()
    train, validate, test = np.split(patients.sample(frac=1, random_state=123), [int(.9*len(patients)), int(.9*len(patients))])
    train_ids = icu_pat['SUBJECT_ID'].isin(train).values
    test_ids = icu_pat['SUBJECT_ID'].isin(test).values
    
    data_train = static[train_ids, :]
    data_test = static[test_ids, :]
    
    label_train = label[train_ids]
    label_test = label[test_ids]  
    
    # Patients in test data
    test_ids_patients = pd.read_pickle(data_dir + 'test_ids_patients.pkl')
    patients = test_ids_patients.drop_duplicates()
    num_patients = patients.shape[0]
    row_ids = pd.DataFrame({'ROW_IDX': test_ids_patients.index}, index=test_ids_patients)
    
    print('-----------------------------------------')  
    
    # Fit logistic regression model
    print('Fit logistic regression model...')
    regr = linear_model.LogisticRegression()
    regr.fit(data_train, label_train)
    
    # BootstrappingFhp.
    np.random.seed(np_seed)
    avpre_vec = np.zeros(bootstrap_samples)
    auroc_vec = np.zeros(bootstrap_samples)
    f1_vec    = np.zeros(bootstrap_samples)
    sensitivity_vec = np.zeros(bootstrap_samples)
    specificity_vec = np.zeros(bootstrap_samples)
    ppv_vec = np.zeros(bootstrap_samples)
    npv_vec = np.zeros(bootstrap_samples)  
    
    for sample in range(bootstrap_samples):
        print('Bootstrap sample {}'.format(sample))
        
        sample_patients = patients.sample(n=num_patients, replace=True)
        idx = np.squeeze(row_ids.loc[sample_patients].values)
        data_test_bs, label_test_bs = data_test[idx], label_test[idx]
        
        label_sigmoids = regr.predict_proba(data_test_bs)[:, 1]
        
        print('Evaluate...')
        # Average precision
        avpre = average_precision_score(label_test_bs, label_sigmoids)
        
        # Determine AUROC score
        auroc = roc_auc_score(label_test_bs, label_sigmoids)
        
        # Sensitivity, specificity
        fpr, tpr, thresholds = roc_curve(label_test_bs, label_sigmoids)
        youden_idx = np.argmax(tpr - fpr)
        sensitivity = tpr[youden_idx]
        specificity = 1-fpr[youden_idx]
        
        # F1, PPV, NPV score
        f1 = 0
        ppv = 0
        npv = 0
        for t in thresholds:
            label_pred = (np.array(label_sigmoids) >= t).astype(int)
            f1_temp = f1_score(label_test_bs, label_pred)
            ppv_temp = precision_score(label_test_bs, label_pred, pos_label=1)
            npv_temp = precision_score(label_test_bs, label_pred, pos_label=0)
            if f1_temp > f1:
                f1 = f1_temp
            if (ppv_temp+npv_temp) > (ppv+npv):
                ppv = ppv_temp
                npv = npv_temp
        
    # Store in vectors
    avpre_vec[sample] = avpre
    auroc_vec[sample] = auroc
    f1_vec[sample]    = f1
    sensitivity_vec[sample]  = sensitivity
    specificity_vec[sample]  = specificity
    ppv_vec[sample]  = ppv
    npv_vec[sample]  = npv
    
    avpre_mean = np.mean(avpre_vec)
    avpre_lci, avpre_uci = st.t.interval(0.95, bootstrap_samples-1, loc=avpre_mean, scale=st.sem(avpre_vec))
    auroc_mean = np.mean(auroc_vec)
    auroc_lci, auroc_uci = st.t.interval(0.95, bootstrap_samples-1, loc=auroc_mean, scale=st.sem(auroc_vec))
    f1_mean = np.mean(f1_vec)
    f1_lci, f1_uci = st.t.interval(0.95, bootstrap_samples-1, loc=f1_mean, scale=st.sem(f1_vec))
    ppv_mean = np.mean(ppv_vec)
    ppv_lci, ppv_uci = st.t.interval(0.95, bootstrap_samples-1, loc=ppv_mean, scale=st.sem(ppv_vec))
    npv_mean = np.mean(npv_vec)
    npv_lci, npv_uci = st.t.interval(0.95,bootstrap_samples-1, loc=npv_mean, scale=st.sem(npv_vec))  
    sensitivity_mean = np.mean(sensitivity_vec)
    sensitivity_lci, sensitivity_uci = st.t.interval(0.95, bootstrap_samples-1, loc=sensitivity_mean, scale=st.sem(sensitivity_vec))
    specificity_mean = np.mean(specificity_vec)
    specificity_lci, specificity_uci = st.t.interval(0.95, bootstrap_samples-1, loc=specificity_mean, scale=st.sem(specificity_vec))
    
    # print('------------------------------------------------')
    # print('Net variant: logistic regression')
    # print('Average Precision: {} [{},{}]'.format(round(avpre_mean), round(avpre_lci), round(avpre_uci)))
    # print('AUROC: {} [{},{}]'.format(round(auroc_mean), round(auroc_lci), round(auroc_uci)))
    # print('F1: {} [{},{}]'.format(round(f1_mean), round(f1_lci), round(f1_uci)))  
    # print('PPV: {} [{},{}]'.format(round(ppv_mean), round(ppv_lci), round(ppv_uci)))
    # print('NPV: {} [{},{}]'.format(round(npv_mean), round(npv_lci), round(npv_uci)))
    # print('Sensitivity: {} [{},{}]'.format(round(sensitivity_mean), round(sensitivity_lci), round(sensitivity_uci)))
    # print('Specificity: {} [{},{}]'.format(round(specificity_mean), round(specificity_lci), round(specificity_uci)))
    # print('Done')

    results_dict = {}
    # results_dict["Net variant"]= "logistic_regression"
    # results_dict["Average Precision"]=str(round(avpre_mean)) + '  [' + str(round(avpre_lci)) + '  ' + str(round(avpre_uci)) +']'
    # results_dict["AUROC"]= str(round(auroc_mean)) + '  [' + str(round(auroc_lci)) + '  ' + str(round(auroc_uci))+']'
    # results_dict["F1"]= str(round(f1_mean)) + '  [' + str(round(f1_lci)) + '  ' + str(round(f1_uci))+']'
    # results_dict["PPV"]= str(round(ppv_mean)) + '  [' + str(round(ppv_lci)) + '  ' + str(round(ppv_uci))+']'
    # results_dict["NPV"]= str(round(npv_mean)) + '  [' +  str(round(npv_lci)) + '  ' +  str(round(npv_uci))+']'
    # results_dict["Sensitivity"]= str(round(sensitivity_mean)) + '  [' + str(round(sensitivity_lci)) + '  ' + str(round(sensitivity_uci))+']'
    # results_dict["Specificity"]= str(round(specificity_mean)) + '  [' + str(round(specificity_lci)) + '  ' + str(round(specificity_uci))+']'
    # results_dict["Time"]= "" 
    # # str(round(times_mean)) + '  ' + str(round(times_lci)) + '  [' + str(round(times_uci))+ '  ' + str(round(times_std))+']'

    results_dict["Net variant"]= "logistic_regression"
    results_dict["Average Precision"]=str(round(avpre_mean)) 
    results_dict["AUROC"]= str(round(auroc_mean)) 
    results_dict["F1"]= str(round(f1_mean)) 
    results_dict["PPV"]= str(round(ppv_mean)) 
    results_dict["NPV"]= str(round(npv_mean))
    results_dict["Sensitivity"]= str(round(sensitivity_mean)) 
    results_dict["Specificity"]= str(round(specificity_mean)) 
    results_dict["Time"]= ""
    # str(round(times_mean)) 


    return results_dict

### Training Code

Training each model uses the data_arrays.npz file generated from the preprocessing steps discussed earlier in this notebook.  This file is approximately 430MB, so it was too large to upload to GitHub.  Training each model for 80 epochs, with most taking approximately 6-8 hours (i.e 4.5-6 minutes/epoch) running from a terminal in a M2 16GB Macbook Pro while the MCE-based models took 1-1.5 hours (~1 minute/epoch).  Total training time took between 70 - 80 hours among the 14 models we trained.  The implementation in the section below largely adheres to the implementation from the orignal paper's repo as well.

* train - function that trains the model passed and captures the results in the "logdir" folder
* test  - function that tests the trained model and captures the performance metrics used for evaluation in the "results" folder.

Training hyperparameters include the following:
* Number of epochs used (80 by default)
* Batch size when loading data (128 by default)
* Dropout rate (0.5 by default)
* Learning rate (0.001 by default)


In [114]:
def num_static(data):
  return data['static_vars'].shape[0]

def vocab_sizes(data):
  return data['dp'].max()+1, data['cp'].max()+1

def abs_time_to_delta(times):
  delta = torch.cat((torch.unsqueeze(times[:, 0], dim=-1), times[:, 1:] - times[:, :-1]), dim=1)
  delta = torch.clamp(delta, min=0)
  return delta

device = 'cpu'

print('Load data...')
current_dir = os.getcwd()
data = np.load(current_dir + '/data/data_arrays.npz', allow_pickle=True)
# Vocabulary sizes
num_static = num_static(data)
num_dp_codes, num_cp_codes = vocab_sizes(data)

def train(net,model_name, num_epochs):
    trainloader, num_batches, pos_weight = get_trainloader(data, 'TRAIN')

     
    
    print('-----------------------------------------')
    print('Start Train...' + model_name)

    # Loss function and optimizer
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(net.parameters(), lr = 0.001)  
    
    # Create log dir
    logdir = current_dir + '/logdir/' + model_name + '/'
    # logdir = hp.logdir + hp.net_variant + '/'
    if not os.path.exists(logdir):
        os.makedirs(logdir)
    
    # Store times
    epoch_times = []
    
    # Train
    for epoch in tqdm(range(num_epochs)): 
    # print('-----------------------------------------')
    # print('Epoch: {}'.format(epoch))
        net.train()
        time_start = time()
        for i, (stat, dp, cp, dp_t, cp_t, label) in enumerate(tqdm(trainloader), 0):
          # move to GPU if available
          stat  = stat.to(device)
          dp    = dp.to(device)
          cp    = cp.to(device)
          dp_t  = dp_t.to(device)
          cp_t  = cp_t.to(device)
          label = label.to(device)
        
          # zero the parameter gradients
          optimizer.zero_grad()
        
          # forward + backward + optimize
          label_pred, _ = net(stat, dp, cp, dp_t, cp_t)
          loss = criterion(label_pred, label)
          loss.backward()
          optimizer.step()
    
    # timing
    time_end = time()
    epoch_times.append(time_end-time_start)
    
    # Save
    print('Saving...')
    torch.save(net.state_dict(), logdir + 'final_model.pt')
    np.savez(logdir + 'epoch_times', epoch_times=epoch_times)
    print('Done Train for ' + model_name)



Load data...


### Testing Code

In [115]:
np_seed = 1234
bootstrap_samples = 1

def num_static1(data):
  return data['static_vars'].shape[0]

def vocab_sizes(data):
  return data['dp'].max()+1, data['cp'].max()+1

def roundval(val):
    if math.isnan(val):
        return"nan"
    else:
        return round(val)
        
def round(num):
  return np.round(num*1000)/1000
    

def test(net, model_name):
    # Load data
    # print('Load data...')
    # # data = np.load(data_dir + 'data_arrays.npz')
    # data = np.load(data_dir + 'data_arrays.npz', allow_pickle=True)
    # # 
    # test_ids_patients = pd.read_pickle(data_dir + 'test_ids_patients.pkl')

    print("Test Evaluation called for " + str(model_name))
    data = np.load(current_dir + '/data/data_arrays.npz', allow_pickle=True)
    test_ids_patients = pd.read_pickle(current_dir + '/data/test_ids_patients.pkl')
    
    # Patients in test data
    patients = test_ids_patients.drop_duplicates()
    num_patients = patients.shape[0]
    row_ids = pd.DataFrame({'ROW_IDX': test_ids_patients.index}, index=test_ids_patients)
    
    # Vocabulary sizes
    num_static = num_static1(data)
    num_dp_codes, num_cp_codes = vocab_sizes(data)
    
    # CUDA for PyTorch
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if use_cuda else 'cpu')
    torch.backends.cudnn.benchmark = True
    
    # Network
    # net = Net(num_static, num_dp_codes, num_cp_codes).to(device)
    
    # Set log dir to read trained model from
    # logdir = logdir + model_name + '/'
    logdir = current_dir + '/logdir/' + model_name + '/'
    print(logdir)
    
    # Restore variables from disk
    net.load_state_dict(torch.load(logdir + 'final_model.pt', map_location=device))
    
    # Bootstrapping
    np.random.seed(np_seed)
    avpre_vec = np.zeros(bootstrap_samples)
    auroc_vec = np.zeros(bootstrap_samples)
    f1_vec    = np.zeros(bootstrap_samples)
    sensitivity_vec = np.zeros(bootstrap_samples)
    specificity_vec = np.zeros(bootstrap_samples)
    ppv_vec = np.zeros(bootstrap_samples)
    npv_vec = np.zeros(bootstrap_samples)

    for sample in range(bootstrap_samples):
        print('Bootstrap sample {}'.format(sample))
        
        # Test data
        sample_patients = patients.sample(n=num_patients, replace=True)
        idx = np.squeeze(row_ids.loc[sample_patients].values)
        testloader, _, _ = get_trainloader(data, 'TEST', shuffle=False, idx=idx)
        
        # evaluate on test data
        net.eval()
        label_pred = torch.Tensor([])
        label_test = torch.Tensor([])
        with torch.no_grad():
            for i, (stat, dp, cp, dp_t, cp_t, label_batch) in enumerate(tqdm(testloader), 0):
                # move to GPU if available
                stat  = stat.to(device)
                dp    = dp.to(device)
                cp    = cp.to(device)
                dp_t  = dp_t.to(device)
                cp_t  = cp_t.to(device)
        
                label_pred_batch, _ = net(stat, dp, cp, dp_t, cp_t)
                label_pred = torch.cat((label_pred, label_pred_batch.cpu()))
                label_test = torch.cat((label_test, label_batch))

        label_sigmoids = torch.sigmoid(label_pred).cpu().numpy()
        
        # Average precision
        avpre = average_precision_score(label_test, label_sigmoids)
        
        # Determine AUROC score
        auroc = roc_auc_score(label_test, label_sigmoids)
        
        # Sensitivity, specificity
        fpr, tpr, thresholds = roc_curve(label_test, label_sigmoids)
        youden_idx = np.argmax(tpr - fpr)
        sensitivity = tpr[youden_idx]
        specificity = 1-fpr[youden_idx]
        
        # F1, PPV, NPV score
        f1 = 0
        ppv = 0
        npv = 0
        for t in thresholds:
            label_pred = (np.array(label_sigmoids) >= t).astype(int)
            f1_temp = f1_score(label_test, label_pred)
            ppv_temp = precision_score(label_test, label_pred, pos_label=1)
            npv_temp = precision_score(label_test, label_pred, pos_label=0)
            if f1_temp > f1:
                f1 = f1_temp
            if (ppv_temp+npv_temp) > (ppv+npv):
                ppv = ppv_temp
                npv = npv_temp

        # print(f1)
        # Store in vectors
        avpre_vec[sample] = avpre
        auroc_vec[sample] = auroc
        f1_vec[sample]    = f1
        sensitivity_vec[sample]  = sensitivity
        specificity_vec[sample]  = specificity
        ppv_vec[sample]  = ppv
        npv_vec[sample]  = npv

    #     print(avpre_vec)
    # print('==')
    # print(avpre_vec)
    avpre_mean = np.mean(avpre_vec)
    # print('avpre_mean')
    # print(avpre_mean)
    
    avpre_lci, avpre_uci = st.t.interval(0.95, bootstrap_samples-1, loc=avpre_mean, scale=st.sem(avpre_vec))
    auroc_mean = np.mean(auroc_vec)
    auroc_lci, auroc_uci = st.t.interval(0.95, bootstrap_samples-1, loc=auroc_mean, scale=st.sem(auroc_vec))
    f1_mean = np.mean(f1_vec)
    f1_lci, f1_uci = st.t.interval(0.95, bootstrap_samples-1, loc=f1_mean, scale=st.sem(f1_vec))
    ppv_mean = np.mean(ppv_vec)
    ppv_lci, ppv_uci = st.t.interval(0.95, bootstrap_samples-1, loc=ppv_mean, scale=st.sem(ppv_vec))
    npv_mean = np.mean(npv_vec)
    npv_lci, npv_uci = st.t.interval(0.95, bootstrap_samples-1, loc=npv_mean, scale=st.sem(npv_vec))
    sensitivity_mean = np.mean(sensitivity_vec)
    sensitivity_lci, sensitivity_uci = st.t.interval(0.95, bootstrap_samples-1, loc=sensitivity_mean, scale=st.sem(sensitivity_vec))
    specificity_mean = np.mean(specificity_vec)
    specificity_lci, specificity_uci = st.t.interval(0.95, bootstrap_samples-1, loc=specificity_mean, scale=st.sem(specificity_vec))
    
    # epoch_times = np.load(logdir + net_variant + '/epoch_times.npz')['epoch_times']
    epoch_times = np.load(logdir +  'epoch_times.npz')['epoch_times']
    # net.load_state_dict(torch.load(logdir + 'final_model.pt', map_location=device))
    times_mean = np.mean(epoch_times)
    times_lci, times_uci = st.t.interval(0.95, len(epoch_times)-1, loc=np.mean(epoch_times), scale=st.sem(epoch_times))
    times_std = np.std(epoch_times)

    # print('------------------------------------------------')
    # print('Net variant: {}'.format(model_name))
    # print('Average Precision: {} [{},{}]'.format(round(avpre_mean), round(avpre_lci), round(avpre_uci)))
    # print('AUROC: {} [{},{}]'.format(round(auroc_mean), round(auroc_lci), round(auroc_uci)))
    # print('F1: {} [{},{}]'.format(round(f1_mean), round(f1_lci), round(f1_uci)))
    # print('PPV: {} [{},{}]'.format(round(ppv_mean), round(ppv_lci), round(ppv_uci)))
    # print('NPV: {} [{},{}]'.format(round(npv_mean), round(npv_lci), round(npv_uci)))
    # print('Sensitivity: {} [{},{}]'.format(round(sensitivity_mean), round(sensitivity_lci), round(sensitivity_uci)))
    # print('Specificity: {} [{},{}]'.format(round(specificity_mean), round(specificity_lci), round(specificity_uci)))
    # print('Time: {} [{},{}] std: {}'.format(round(times_mean), round(times_lci), round(times_uci), round(times_std)))
    # print('Done')

    # results_file = current_dir + '/results/' + model_name + '_results.txt'
    # with open(results_file, 'a') as f:
    #     f.write('\n')
    #     f.write('\nNet variant: {}'.format(model_name))
    #     f.write('\nAverage Precision: {} [{},{}]'.format(round(avpre_mean), round(avpre_lci), round(avpre_uci)))
    #     f.write('\nAUROC: {} [{},{}]'.format(round(auroc_mean), round(auroc_lci), round(auroc_uci)))
    #     f.write('\nF1: {} [{},{}]'.format(round(f1_mean), round(f1_lci), round(f1_uci)))
    #     f.write('\nPPV: {} [{},{}]'.format(round(ppv_mean), round(ppv_lci), round(ppv_uci)))
    #     f.write('\nNPV: {} [{},{}]'.format(round(npv_mean), round(npv_lci), round(npv_uci)))
    #     f.write('\nSensitivity: {} [{},{}]'.format(round(sensitivity_mean), round(sensitivity_lci), round(sensitivity_uci)))
    #     f.write('\nSpecificity: {} [{},{}]'.format(round(specificity_mean), round(specificity_lci), round(specificity_uci)))
    #     f.write('\nTime: {} [{},{}] std: {}'.format(round(times_mean), round(times_lci), round(times_uci), round(times_std)))
    #     f.write('\n Test Complete')

    
    results_dict = {}
    # results_dict["Net variant"]= model_name
    # results_dict["Average Precision"]=str(round(avpre_mean)) + '  [' + str(round(avpre_lci)) + '  ' + str(round(avpre_uci)) +']'
    # results_dict["AUROC"]= str(round(auroc_mean)) + '  [' + str(round(auroc_lci)) + '  ' + str(round(auroc_uci))+']'
    # results_dict["F1"]= str(round(f1_mean)) + '  [' + str(round(f1_lci)) + '  ' + str(round(f1_uci))+']'
    # results_dict["PPV"]= str(round(ppv_mean)) + '  [' + str(round(ppv_lci)) + '  ' + str(round(ppv_uci))+']'
    # results_dict["NPV"]= str(round(npv_mean)) + '  [' +  str(round(npv_lci)) + '  ' +  str(round(npv_uci))+']'
    # results_dict["Sensitivity"]= str(round(sensitivity_mean)) + '  [' + str(round(sensitivity_lci)) + '  ' + str(round(sensitivity_uci))+']'
    # results_dict["Specificity"]= str(round(specificity_mean)) + '  [' + str(round(specificity_lci)) + '  ' + str(round(specificity_uci))+']'
    # results_dict["Time"]= str(round(times_mean)) + '  ' + str(round(times_lci)) + '  [' + str(round(times_uci))+ '  ' + str(round(times_std))+']'


    results_dict["Net variant"]= model_name
    results_dict["Average Precision"]=str(round(avpre_mean)) 
    results_dict["AUROC"]= str(round(auroc_mean)) 
    results_dict["F1"]= str(round(f1_mean)) 
    results_dict["PPV"]= str(round(ppv_mean)) 
    results_dict["NPV"]= str(round(npv_mean))
    results_dict["Sensitivity"]= str(round(sensitivity_mean)) 
    results_dict["Specificity"]= str(round(specificity_mean)) 
    results_dict["Time"]= str(round(times_mean)) 


    

    return results_dict

In [116]:
# num_epochs = 80

# model=birnn_concat_time_delta_attention(num_static, num_dp_codes, num_cp_codes)
# train(model, 'birnn_concat_time_delta',num_epochs)

# model=birnn_ode_decay(num_static, num_dp_codes, num_cp_codes)
# train(model, 'birnn_ode_decay',num_epochs)

# model=birnn_time_decay_attention(num_static, num_dp_codes, num_cp_codes)
# train(model, 'birnn_time_decay_attention',num_epochs)

# model=ode_attention(num_static, num_dp_codes, num_cp_codes)
# train(model, 'ode_attention',num_epochs)

# model=ode_birnn_attention(num_static, num_dp_codes, num_cp_codes)
# train(model, 'ode_birnn_attention',num_epochs)

# model=ode_birnn(num_static, num_dp_codes, num_cp_codes)
# train(model, 'ode_birnn',num_epochs)

# model=attention_concat_time(num_static, num_dp_codes, num_cp_codes)
# train(model, 'attention_concat_time',num_epochs)

# model=birnn_ode_decay_attention(num_static, num_dp_codes, num_cp_codes)
# train(model, 'birnn_ode_decay_attention',num_epochs)

# model=mce_attention(num_static, num_dp_codes, num_cp_codes)
# train(model, 'mce_attention',num_epochs)

# model=mce_birnn(num_static, num_dp_codes, num_cp_codes)
# train(model, 'mce_birnn',num_epochs)

# model=mce_birnn_attention(num_static, num_dp_codes, num_cp_codes)
# train(model, 'mce_birnn_attention',num_epochs)

# Results

According to our paper, the paper's results indicate that the deep learning architectures, particularly those with a recurrent component, outperformed the logistic regression baseline model in predicting ICU readmissions.

Baseline Characteristics: The analysis involved 23 static variables, 992 unique ICD-9 diagnosis codes, 298 unique ICD-9 procedure codes, 586 unique medications, and 32 codes related to vital signs. Each patient's electronic medical record (EMR) contained a maximum of 552 ICD-9 diagnosis and procedure codes and 392 medications and vital sign codes associated with the current ICU stay.

Performance Trends: Models with a recurrent component generally performed better (average precision range: 0.298–0.331) than those based solely on attention layers (average precision range: 0.269–0.294). This suggests that incorporating recurrent neural network (RNN) components improved the predictive power for ICU readmissions.

Therefore, the deep learning architectures, especially those combining recurrent components, showed significantly improved predictive accuracy compared to traditional logistic regression models when predicting ICU readmissions.

Below are the tables of our results from each model we tested.
1. <b>Results of model from the paper</b>: These are the base results from the paper itself.<ul>display_originalpaper_results()
    </ul>
   
2. <b>Results of our training model</b> : These are the results based on running our reproduced trained models for bootstrap_samples = 100 similar to the paper.<ul>display_reproduced_results()</ul>  
   
3. <b>Results for quick evaluation and validation</b> : Knowing that the full run takes time, we have provided an alternate version, with bootstrap_samples =1 that can be run if needed as part of the evaluation.<ul>eval_shorter_run()<br>display_shorter_run_results()</ul> 


In [117]:
def eval_shorter_run():
    current_dir = os.getcwd()
    all_results={}
    
    model=ode_birnn_attention(num_static, num_dp_codes, num_cp_codes)
    all_results["ode_birnn_attention"]=test(model, 'ode_birnn_attention')
    
    model=ode_birnn(num_static, num_dp_codes, num_cp_codes)
    all_results["ode_birnn"]=test(model, 'ode_birnn')
    
    model=birnn_ode_decay_attention(num_static, num_dp_codes, num_cp_codes)
    all_results["birnn_ode_decay_attention"]=test(model, 'birnn_ode_decay_attention')
    
    model=birnn_ode_decay(num_static, num_dp_codes, num_cp_codes)
    all_results["birnn_ode_decay"]=test(model, 'birnn_ode_decay')
    
    model=birnn_time_decay_attention(num_static, num_dp_codes, num_cp_codes)
    all_results["birnn_time_decay_attention"]=test(model, 'birnn_time_decay_attention')
    
    model=birnn_time_decay(num_static, num_dp_codes, num_cp_codes)
    all_results["birnn_time_decay"]=test(model, 'birnn_time_decay')
    
    model=birnn_concat_time_delta_attention(num_static, num_dp_codes, num_cp_codes)
    all_results["birnn_concat_time_delta_attention"]=test(model, 'birnn_concat_time_delta_attention')
    
    model=birnn_concat_time_delta(num_static, num_dp_codes, num_cp_codes)
    all_results["birnn_concat_time_delta"]=test(model, 'birnn_concat_time_delta')
    
    model=ode_attention(num_static, num_dp_codes, num_cp_codes)
    all_results["ode_attention"]=test(model, 'ode_attention')
    
    model=attention_concat_time(num_static, num_dp_codes, num_cp_codes)
    all_results["attention_concat_time"]=test(model, 'attention_concat_time')
    
    model=mce_birnn_attention(num_static, num_dp_codes, num_cp_codes)
    all_results["mce_birnn_attention"]=test(model, 'mce_birnn_attention')
    
    model=mce_birnn(num_static, num_dp_codes, num_cp_codes)
    all_results["mce_birnn"]=test(model, 'mce_birnn')
    
    model=mce_attention(num_static, num_dp_codes, num_cp_codes)
    all_results["mce_attention"]=test(model, 'mce_attention')
    
    all_results["logistic_regression"]=logistic_regression(1,1234)
    
    sr_file = current_dir + '/results/shortresults.pickle'
    
    file = open(sr_file, 'wb')
    pickle.dump(all_results, file)
    file.close()

    
    


In [127]:
def display_shorter_run_results():
    current_dir = os.getcwd()
    shortresults_data = current_dir + '/results/shortresults.pickle'
    
    file = open(shortresults_data, 'rb')
    all_results = pickle.load(file)
    
     
    headers = ["Net variant", "Average Precision", "AUROC", "F1", "Sensitivity", "Specificity"]
    
    data = []
    for result in all_results.values():
        data.append([result[key] for key in headers])
    
    print("Results of our training model")
    print(tabulate(data, headers=headers,tablefmt='presto',numalign="center"))

    

In [119]:
def display_originalpaper_results():
    
    results = {
        "ODE + RNN + Attention": {
            "Net variant": "ODE + RNN + Attention",
            "Average Precision": "0.314 ",
            "AUROC": "0.739 ",
            "F1": "0.376 ",
            "Sensitivity": "0.685 ",
            "Specificity": "0.677 ",
        },
    
        "ODE + RNN": {
            "Net variant": "ODE + RNN",
            "Average Precision": "0.331 ",
            "AUROC": "0.739 ",
            "F1": "0.372 ",
            "Sensitivity": "0.672",
            "Specificity": "0.697 ",
        },
        "RNN (ODE time decay) + Attention": {
            "Net variant": "RNN (ODE time decay) + Attention",
            "Average Precision": "0.316 ",
            "AUROC": "0.743",
            "F1": "0.375 ",
            "Sensitivity": "0.648 ",
            "Specificity": "0.733 ",
        },
        "RNN (ODE time decay)": {
            "Net variant": "RNN (ODE time decay)",
            "Average Precision": "0.300 ",
            "AUROC": "0.741 ",
            "F1": "0.372 ",
            "Sensitivity": "0.710 ",
            "Specificity": "0.667 ",
        },
        "RNN (exp time decay) + Attention": {
            "Net variant": "RNN (exp time decay) + Attention",
            "Average Precision": "0.320 ",
            "AUROC": "0.748 ",
            "F1": "0.377  ",
            "Sensitivity": "0.704 ",
            "Specificity": "0.680 ",
        },
        "RNN (exp time decay)": {
            "Net variant": "RNN (exp time decay)",
            "Average Precision": "0.304 ",
            "AUROC": "0.735",
            "F1": "0.368 ",
            "Sensitivity": "0.707 ",
            "Specificity": "0.670 ",
        },
        "RNN (concatenated Δtime) + Attention": {
            "Net variant": "RNN (concatenated Δtime) + Attention",
            "Average Precision": "0.312 ",
            "AUROC": "0.741 ",
            "F1": "0.368 ",
            "Sensitivity": "0.687 ",
            "Specificity": "0.688 ",
        },
        "RNN (concatenated Δtime)": {
            "Net variant": "RNN (concatenated Δtime)",
            "Average Precision": "0.311 ",
            "AUROC": "0.739 ",
            "F1": "0.364 ",
            "Sensitivity": "0.698 ",
            "Specificity": "0.688 ",
        },
        "ODE + Attention": {
            "Net variant": "ODE + Attention",
            "Average Precision": "0.294 ",
            "AUROC": "0.717 ",
            "F1": "0.333 ",
            "Sensitivity": "0.776 ",
            "Specificity": "0.554 ",
        },
        "Attention (concatenated time)": {
            "Net variant": "Attention (concatenated time)",
            "Average Precision": "0.286 ",
            "AUROC": "0.711 ",
            "F1": "0.330 ",
            "Sensitivity": "0.700 ",
            "Specificity": "0.614 ",
        },
        "MCE + RNN + Attention": {
            "Net variant": "MCE + RNN + Attention",
            "Average Precision": " 0.317 ",
            "AUROC": "0.736",
            "F1": "0.373",
            "Sensitivity": "0.630 ",
            "Specificity": "0.744",
        },
        "MCE + RNN": {
            "Net variant": "MCE + RNN",
            "Average Precision": "0.298",
            "AUROC": "0.727 ",
            "F1": "0.361",
            "Sensitivity": "0.654 ",
            "Specificity": "0.706 ",
        },
        "MCE + Attention": {
            "Net variant": "MCE + Attention",
            "Average Precision": " 0.269 ",
            "AUROC": "0.689 ",
            "F1": "0.312 ",
            "Sensitivity": "0.686 ",
            "Specificity": "0.616 ",
        },
        "Logistic Regression": {
            "Net variant": "Logistic Regression",
            "Average Precision": "0.257 ",
            "AUROC": "0.659 ",
            "F1": "0.296 ",
            "Sensitivity": "0.606 ",
            "Specificity": "0.647 ",
        },
    }
    
    headers = ["Net variant", "Average Precision", "AUROC", "F1", "Sensitivity", "Specificity"]
    
    data = []
    for result in results.values():
        data.append([result[key] for key in headers])
    
    print("Results of model from the paper")
    print(tabulate(data, headers=headers))

In [120]:
def display_reproduced_results():
    current_dir = os.getcwd()
    all_results={}
    
    model_results = current_dir + '/results/ode_birnn_attention.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["ode_birnn_attention"]=result
    file.close
    
    model_results = current_dir + '/results/ode_birnn.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["ode_birnn"]=result
    file.close
    
    
    model_results = current_dir + '/results/birnn_ode_decay_attention.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["birnn_ode_decay_attention"]=result
    file.close
    
    
    model_results = current_dir + '/results/birnn_ode_decay.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["birnn_ode_decay"]=result
    file.close
    
    model_results = current_dir + '/results/birnn_time_decay_attention.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["birnn_time_decay_attention"]=result
    file.close
    
    model_results = current_dir + '/results/birnn_time_decay.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["birnn_time_decay"]=result
    file.close
    
    model_results = current_dir + '/results/birnn_concat_time_delta_attention.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["birnn_concat_time_delta_attention"]=result
    file.close
    
    
    model_results = current_dir + '/results/birnn_concat_time_delta.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["birnn_concat_time_delta"]=result
    file.close
    
    
    model_results = current_dir + '/results/ode_attention.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["ode_attention"]=result
    file.close
    
    
    model_results = current_dir + '/results/attention_concat_time.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["attention_concat_time"]=result
    file.close
    
    
    
    model_results = current_dir + '/results/mce_birnn_attention.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["mce_birnn_attention"]=result
    file.close
    
    model_results = current_dir + '/results/mce_birnn.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["mce_birnn"]=result
    file.close
    
    model_results = current_dir + '/results/mce_attention.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["mce_attention"]=result
    file.close
    
    model_results = current_dir + '/results/logistic_regression.pickle'
    file = open(model_results, 'rb')
    result = pickle.load(file)
    all_results["logistic_regression"]=result
    file.close
    
    
    headers = ["Net variant", "Average Precision", "AUROC", "F1",  "Sensitivity", "Specificity"]
    
    data = []
    for result in all_results.values():
        data.append([result[key] for key in headers])
    
    print("Results of our training model")
    print(tabulate(data, headers=headers,tablefmt='presto',numalign="center"))


In [126]:
print("-----------------------------------------------------------------------------------------------------------")
display_originalpaper_results()
print("-----------------------------------------------------------------------------------------------------------")
display_reproduced_results()

-----------------------------------------------------------------------------------------------------------
Results of model from the paper
Net variant                             Average Precision    AUROC     F1    Sensitivity    Specificity
------------------------------------  -------------------  -------  -----  -------------  -------------
ODE + RNN + Attention                               0.314    0.739  0.376          0.685          0.677
ODE + RNN                                           0.331    0.739  0.372          0.672          0.697
RNN (ODE time decay) + Attention                    0.316    0.743  0.375          0.648          0.733
RNN (ODE time decay)                                0.3      0.741  0.372          0.71           0.667
RNN (exp time decay) + Attention                    0.32     0.748  0.377          0.704          0.68
RNN (exp time decay)                                0.304    0.735  0.368          0.707          0.67
RNN (concatenated Δtime) + Att

## Summary of Results

Across all models, we obtained similar results to the paper for average precision, AUROC, and F1.  Sensitivity and specificity measures, however, tended to vary more from the paper's findings.  The results from our trained models reinforced the fundamental hypotheses of the paper, which were that all deep learning models performed better than the baseline logistic regression, and that models that used an RNN (birnn in the model names above) tended to perform most accurately.  The ablation study was implicit in the design of the original paper--combinations of deep learning approaches such as RNN, MCE, ODE, and attention were used to build separate models, and some models could be directly compared to others such as MCE + RNN vs. MCE + RNN + Attention.  Models with attention tended to perform better against their counterparts that did not use it, with the exception of the ODE + RNN models.

In our replication attempts, we did not choose to pursue experiments beyond the original paper.

In [124]:
# This cell and the one below it can be run to see a shorter run evaluation with one bootstrap sample against our training models.
eval_shorter_run()

Test Evaluation called for ode_birnn_attention
/Users/austinwang/Documents/Graduate School/Deep Learning For Healthcare (CS 598)/Project Code/DLH_Project/logdir/ode_birnn_attention/
Bootstrap sample 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:10<00:00,  3.42it/s]


Test Evaluation called for ode_birnn
/Users/austinwang/Documents/Graduate School/Deep Learning For Healthcare (CS 598)/Project Code/DLH_Project/logdir/ode_birnn/
Bootstrap sample 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:09<00:00,  3.66it/s]


Test Evaluation called for birnn_ode_decay_attention
/Users/austinwang/Documents/Graduate School/Deep Learning For Healthcare (CS 598)/Project Code/DLH_Project/logdir/birnn_ode_decay_attention/
Bootstrap sample 0


...

Test Evaluation called for mce_birnn
/Users/austinwang/Documents/Graduate School/Deep Learning For Healthcare (CS 598)/Project Code/DLH_Project/logdir/mce_birnn/
Bootstrap sample 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:02<00:00, 12.67it/s]


Test Evaluation called for mce_attention
/Users/austinwang/Documents/Graduate School/Deep Learning For Healthcare (CS 598)/Project Code/DLH_Project/logdir/mce_attention/
Bootstrap sample 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:01<00:00, 30.73it/s]


/Users/austinwang/Documents/Graduate School/Deep Learning For Healthcare (CS 598)/Project Code/DLH_Project/data/
Loading data...
Loading last vital signs measurements...
-----------------------------------------
Create array of static variables...
Create static array...
Create label array...
-----------------------------------------
Split data into train/validate/test...
-----------------------------------------
Fit logistic regression model...
Bootstrap sample 0
Evaluate...


In [128]:
display_shorter_run_results()

Results of our training model
 Net variant                       |  Average Precision  |  AUROC  |  F1   |  Sensitivity  |  Specificity
-----------------------------------+---------------------+---------+-------+---------------+---------------
 ode_birnn_attention               |        0.317        |  0.721  | 0.368 |     0.496     |     0.825
 ode_birnn                         |        0.33         |  0.72   | 0.388 |     0.58      |     0.776
 birnn_ode_decay_attention         |        0.323        |  0.724  | 0.38  |     0.489     |     0.844
 birnn_ode_decay                   |        0.317        |  0.724  | 0.375 |     0.587     |     0.757
 birnn_time_decay_attention        |        0.309        |  0.731  | 0.372 |     0.655     |     0.696
 birnn_time_decay                  |        0.314        |  0.72   | 0.368 |     0.674     |     0.69
 birnn_concat_time_delta_attention |        0.298        |  0.718  | 0.351 |     0.598     |     0.722
 birnn_concat_time_delta           |

# Discussion

As mentioned in the summary of results section, precision, AUROC, and F1 of our trained models were close to those that were presented in the paper, while sensitivity and specificity measures varied. However, since the hypothesis of the paper revolved around the first three measures, we found the paper is reproducible and came to the same conclusions that the paper presented.<br>

Overall, reproducing the results of the paper was generally easy, with the vast majority of code being usable as is and the underlying dataset (MIMIC-III) being publicly available. The main part that was difficult to reproduce was the sensitivity and specificity numbers.  There was not much detail on how these numbers were computed in the paper, and executing the code in the original GitHub repo simply gave us different results. In addition, it would have been more straightforward if the authors had included a requirements.txt file with their code, which would have provided benchmark library versions to make replication more predictable.  Mentioning the version of Python used in the original paper would have helped us debug and investigate any result inconsistencies. It was up to us to figure out how to run the preprocessing steps; had the authors provided the dependencies and order to run the preprocessing steps, that would have helped us get set up more quickly.  Lastly, training most models took 6-8 hours of time on a personal computer due to having only CPU available and not GPU.

The main thing that would have helped us in reproducing the original paper's results is better code documentation and a README file that instructs the user on potential code dependencies.  While we don't expect the paper to cover such implementation details, some documentation on the formulas used and how to run the scripts would have reduced the chance for human error.  That is not to say that the code was undocumented; the models were well-documented and helped us understand how they were being set up.



# References
1.  “Benchmarking Deep Learning Architectures for Predicting Readmission to the ICU and Describing Patients-at-Risk”, Sebastiano Barbieri, James Kemp, 2020

